# skenario 1 (ML)

## Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nolimit/NER/data/data_train_ner.csv')

In [ ]:
df.sample(5)

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.drop(df[df.tag == 'O1'].index, inplace=True)
df.drop(df[df.tag == 'OO'].index, inplace=True)

In [ ]:
df.tag.value_counts()

In [ ]:
#Simple feature map to feed arrays into the classifier.
def feature_map(word):
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [ ]:
df['word'] = df['word'].astype(str)

In [ ]:
#We divide the dataset into train and test sets
words = [feature_map(w) for w in df["word"].values.tolist()]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

tags = le.fit_transform(df["tag"].values.tolist())

In [ ]:
X = words
y = tags

classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=0)

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

#Lets check the performance
from sklearn.metrics import classification_report
report = classification_report(y_pred=y_pred, y_true=y_test)
print(report)

In [ ]:
tes = [feature_map(w) for w in ['ilmu']]

model.predict(tes)

## Train with imbalanced

In [ ]:
# Define a pipeline combining a text feature extractor with multi lable classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SVMSMOTE

sm = SVMSMOTE(random_state=42)

X = words
y = tags

# classes = np.unique(y)
# classes = classes.tolist()

X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.30, random_state=0)

model = OneVsRestClassifier(MultinomialNB())
# model = BalancedBaggingClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

#Lets check the performance
from sklearn.metrics import classification_report
report = classification_report(y_pred=y_pred, y_true=y_test)
print(report)

In [ ]:
# # save the model to disk
# filename = 'model_ner.pkl'
# pickle.dump(model, open(filename, 'wb'))

# skenario 2 (DL)

## Import, pre-processing

In [ ]:
# !pip uninstall gensim

In [ ]:
!pip install gensim

In [ ]:
# Read in the data and clean up column names

import gensim

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nolimit/NER/data/data_train_ner.csv')

In [ ]:
df['word'] = df['word'].astype(str)

In [ ]:
# Clean data using the built in cleaner in gensim
df['word_clean'] = df['word'].str.replace(r'[^\w\s]+', '')
df['word_clean'] = df['word_clean'].str.lower()
df.head()

In [ ]:
df.drop(df[df.tag == 'O1'].index, inplace=True)
df.drop(df[df.tag == 'OO'].index, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.isna().sum()

In [ ]:
# df['tag'] = df['tag'].str.strip('I-')
# df['tag'] = df['tag'].str.strip('B-')

In [ ]:
# df.tag.value_counts()

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df[df['tag'] == 'B-NUM']

In [ ]:
df.tag.value_counts()

In [ ]:
df.to_csv('/content/drive/MyDrive/nolimit/NER/data/data_clean.csv', index=False)

## word2vec - skenario 1 (gagal)

In [ ]:
# Encoding the label column
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df['tag_num'] = le.fit_transform(df["tag"].values.tolist())

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['word_clean'], df['tag_num'] , test_size=0.2)

In [ ]:
# 8 = TL
# 7 = PRO
# 6 = PER
# 5 = ORG
# 4 = O
# 3 = NUM
# 2 = LOC
# 1 = EVE
# 0 = DT

In [ ]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   window=5,
                                   min_count=2)

In [ ]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('jakarta')

In [ ]:
w2v_model.wv.index_to_key

In [ ]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [ ]:
# Why is the length of the sentence different than the length of the sentence vector?
# for i, v in enumerate(X_train_vect):
#     print(len(X_train.iloc[i]), len(v))

In [ ]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

## pad sequence - skenario 2

In [ ]:
# Read in the data and clean up column names

import gensim

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nolimit/NER/data/data_clean.csv')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df['word_clean'] = df['word_clean'].astype(str)

## Pra Pengolahan - Tokenisasi dan Sequence
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['word_clean'])
sequences = tokenizer.texts_to_sequences(df['word_clean'])
words = pad_sequences(sequences, maxlen=50, padding='post')

In [ ]:
# Encoding the label column
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df['tag_num'] = le.fit_transform(df["tag"].values.tolist())

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(words, df['tag_num'] , test_size=0.2)

In [ ]:
print(df[['tag', 'tag_num']].drop_duplicates(subset=['tag_num']).sort_values(by=['tag_num']))

## modelling ML

In [ ]:
df.tag.value_counts().plot(kind='bar')

In [ ]:
# # pakai ini jika mau coba balance
# from imblearn.ensemble import BalancedRandomForestClassifier
# clf = BalancedRandomForestClassifier()
# rf_model = clf.fit(X_train, y_train.values.ravel())

In [ ]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model = rf.fit(X_train, y_train.values.ravel())

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
model = clf.fit(X_train, y_train.values.ravel())

In [ ]:
# Use the trained model to make predictions on the test data
y_pred = model.predict(X_test)

In [ ]:
len(np.unique(y_pred))

In [ ]:
len(y_test.value_counts())

In [ ]:
#Lets check the performance
from sklearn.metrics import classification_report
report = classification_report(y_pred=y_pred, y_true=y_test)
print(report)

In [ ]:
# save the model to disk

import pickle

filename = '/content/drive/MyDrive/nolimit/NER/model_bi_rf.pkl'
pickle.dump(rf_model, open(filename, 'wb'))

## load model

In [ ]:
import pickle

model = pickle.load(open('/content/drive/MyDrive/nolimit/NER/model_bi_rf.pkl', 'rb'))

tokenizer = pickle.load(open('/content/drive/MyDrive/nolimit/NER/tokenizer.pkl', 'rb'))

In [ ]:
# #Lets check the performance
# from sklearn.metrics import classification_report

# y_pred = model.predict(X_test)
# report = classification_report(y_pred=y_pred, y_true=y_test)
# print(report)

In [ ]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

from nltk.util import ngrams

from nltk.corpus import stopwords
sw = stopwords.words("indonesian")

def text_cleaning(text, remove_stop_words=True):
    # Clean the text

    text = text.lower() #lowercase atau case folding
    text = re.sub('@[^\s]+', '', text) #remove username
    text = re.sub('\[.*?\]', '', text) # remove square brackets
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text) # remove URLs
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)

    # Optionally, remove stop words
    if remove_stop_words:
        # load stopwords
        text = text.split()
        text = [w for w in text if not w in sw] #6
        text = " ".join(text)
        text = text.lower()

    return text

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# text_new = """
# Sampai 31 Oktober 2022 jumlah kasus (gagal ginjal akut) ada 304, dirawat sebanyak 46 kasus, meninggal 159 kasus atau 52 persen, dan yang sembuh 99 orang, kata Juru Bicara Kemenkes Mohammad Syahril dalam konferensi pers daring di Jakarta, Selasa (1/11/2022)
# """

text_new = "Muhammad Fahmi pergi ke mall taman anggrek membeli kue bantal"

# text_new = """JAKARTA, KOMPAS.com - Menteri Kesehatan (Menkes) Budi Gunadi Sadikin memaparkan,
# jumlah total kasus gagal ginjal akut di Indonesia mencapai 325 kasus per Selasa (1/11/2022)
# """

cleaned_review = text_cleaning(text_new)
tokens = cleaned_review.lower().split()
# bigrm = [' '.join(e) for e in ngrams(tokens, 2)]

# tokens.extend(bigrm)

## Pra Pengolahan Data Baru
sequences = tokenizer.texts_to_sequences(tokens)
vec_tokens = pad_sequences(sequences, maxlen=50, padding='post')

In [ ]:
# prediksi
hasil = model.predict(vec_tokens)

hasil_prediksi = ['B-DT' if i == 0 else
                  'B-EVE' if i == 1 else
                  'B-LOC' if i == 2 else
                  'B-NUM' if i == 3 else
                  'B-ORG' if i == 4 else
                  'B-PER' if i == 5 else
                  'B-PRO' if i == 6 else
                  'B-TL' if i == 7 else
                  'I-DT' if i == 8 else
                  'I-EVE' if i == 9 else
                  'I-LOC' if i == 10 else
                  'I-NUM' if i == 11 else
                  'I-ORG' if i == 12 else
                  'I-PER' if i == 13 else
                  'I-PRO' if i == 14 else
                  'I-TL' if i == 15 else
                  'O' if i == 16 else
                  i for i in hasil]

In [ ]:
for a,b in zip(tokens, hasil_prediksi):
  print(a, '=>', b)

In [ ]:
dt = []
dh = []
statement = {}

for t,h in zip(tokens, hasil_prediksi):
  # print(t, '-->', h)
  dt.append(t)
  dh.append(h)

  if h == 'PER':
    # print(type(t))
    # print(t, 'mengatakan : ', ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]))
    # statement = {t : ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]))}

    if bool(re.search(r"kata", text_new.lower())) == True:
      statement[t] = ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]).strip()

    elif bool(re.search(r"memaparkan", text_new.lower())) == True:
      statement[t] = text_new.lower().split('memaparkan,')[1].strip()


output = {dt[i]: dh[i] for i in range(len(dt))}
print(text_new.lower())
print(statement)

In [ ]:
# Sampai 31 Oktober 2022 jumlah kasus (gagal ginjal akut) ada 304,
# dirawat sebanyak 46 kasus, meninggal 159 kasus atau 52 persen,
# dan yang sembuh 99 orang, kata Juru Bicara Kemenkes Mohammad Syahril
# dalam konferensi pers daring di Jakarta, Selasa (1/11/2022)

In [ ]:
result = {"prediction":output, "statement":statement}
result

In [ ]:
# # tes bigram

# import nltk
# text = 'Sampai 31 Oktober 2022 jumlah kasus (gagal ginjal akut) ada 304, dirawat sebanyak 46 kasus, meninggal 159 kasus atau 52 persen, dan yang sembuh 99 orang," kata Juru Bicara Kemenkes Mohammad Syahril dalam konferensi pers daring di Jakarta, Selasa (1/11/2022)'

# bigrm = list(nltk.bigrams(text.split()))
# bigrm

## Compare Model

In [ ]:
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [ ]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(words, df['tag_num'] , test_size=0.2)

In [ ]:
models = SVC()
models.fit(X_train, y_train)

In [ ]:
results = []
names = []
scoring = 'accuracy'
for name, model in models:
  kfold = model_selection.KFold(n_splits=10)
  cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

# IndoBert

## import

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/williamtjhi/SEA_NLP_Workshop.git

Cloning into 'SEA_NLP_Workshop'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 56 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), 6.48 MiB | 3.54 MiB/s, done.


In [ ]:
%cd /content/SEA_NLP_Workshop

/content/SEA_NLP_Workshop


In [ ]:
!pip install -r requirements_IndoBERT.txt

In [ ]:
import numpy as np
import pandas as pd
import torch

from transformers import BertConfig, BertTokenizer #need transformers==2.9.0
from nltk.tokenize import word_tokenize

from word_classification import BertForWordClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')

Downloading:   0%|          | 0.00/229k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [ ]:
config.num_labels = 7
i2w = {0: 'I-PERSON', 1: 'B-ORGANISATION', 2: 'I-ORGANISATION', 3: 'B-PLACE', 4: 'I-PLACE', 5: 'O', 6: 'B-PERSON'}

## Finetunning

In [ ]:
%cd /content/SEA_NLP_Workshop

/content/SEA_NLP_Workshop/Datasets


In [ ]:
!git clone https://github.com/IndoNLP/indonlu

Cloning into 'indonlu'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 500 (delta 115), reused 142 (delta 111), pack-reused 316
Receiving objects: 100% (500/500), 9.45 MiB | 11.03 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [ ]:
import os, sys
sys.path.append('../')
os.chdir('../')

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
from tqdm import tqdm

from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

%cd /content/SEA_NLP_Workshop/indonlu
from word_classification import BertForWordClassification
from utils.forward_fn import forward_word_classification
from utils.metrics import ner_metrics_fn
from utils.data_utils import NerGritDataset, NerDataLoader

import nltk
nltk.download('punkt')

/content/SEA_NLP_Workshop/indonlu


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)


In [ ]:
# Set random seed
set_seed(26092020)

In [ ]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1', num_hidden_layers = 6)
config.num_labels = NerGritDataset.NUM_LABELS

# Instantiate model
model = BertForWordClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [ ]:
# model

In [ ]:
count_param(model)

81919495

In [ ]:
config.num_hidden_layers

6

## combine dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nolimit/NER/data/data_train_ner.csv')

In [ ]:
df.head()

,word,tag
0,JAKARTA,B-LOC
1,–,O
2,Tanah,O
3,Air,O
4,kini,O


In [ ]:
df['tag'].value_counts()

O        4065887
I-DT      121960
B-PER     107586
B-LOC      93886
I-ORG      86435
I-NUM      75037
I-LOC      70380
B-ORG      68166
I-TL       62389
I-PER      61130
B-NUM      46418
B-DT       41827
I-EVE      36276
B-TL       33543
B-EVE      17041
I-PRO       8903
B-PRO       6766
O1             6
OO             2
Name: tag, dtype: int64

In [ ]:
drop = df[ (df['tag'] == 'I-DT') | (df['tag'] == 'I-NUM') | (df['tag'] == 'I-TL') |
           (df['tag'] == 'B-NUM') | (df['tag'] == 'B-DT') | (df['tag'] == 'I-EVE') |
           (df['tag'] == 'B-TL') | (df['tag'] == 'B-EVE') | (df['tag'] == 'I-PRO') |
           (df['tag'] == 'B-PRO') | (df['tag'] == 'O1') | (df['tag'] == 'OO')].index

df.drop(drop , inplace=True)

In [ ]:
df['tag'].value_counts()

O        4065887
B-PER     107586
B-LOC      93886
I-ORG      86435
I-LOC      70380
B-ORG      68166
I-PER      61130
Name: tag, dtype: int64

In [ ]:
# 'I-PERSON': 0, 'B-ORGANISATION': 1, 'I-ORGANISATION': 2, 'B-PLACE': 3, 'I-PLACE': 4, 'O': 5, 'B-PERSON': 6

df = df.replace(to_replace="B-PER", value="B-PERSON")
df = df.replace(to_replace="B-LOC", value="B-PLACE")
df = df.replace(to_replace="I-ORG", value="I-ORGANISATION")
df = df.replace(to_replace="I-LOC", value="I-PLACE")
df = df.replace(to_replace="B-ORG", value="B-ORGANISATION")
df = df.replace(to_replace="I-PER", value="I-PERSON")

In [ ]:
df['tag'].value_counts()

O                 4065887
B-PERSON           107586
B-PLACE             93886
I-ORGANISATION      86435
I-PLACE             70380
B-ORGANISATION      68166
I-PERSON            61130
Name: tag, dtype: int64

In [ ]:
df.duplicated().sum()

4566629

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df['tag'].value_counts()

O                 17807
B-PERSON           1621
I-PERSON           1608
I-ORGANISATION     1323
I-PLACE            1291
B-PLACE             966
B-ORGANISATION      817
Name: tag, dtype: int64

In [ ]:
df.word.isna().sum()

1

In [ ]:
df = df.dropna(subset=['word']).reset_index(drop=True)

In [ ]:
df[df.word.isna()]

,word,tag


In [ ]:
df = df[df["word"].str.strip() != ""]

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df

,word,tag
0,JAKARTA,B-PLACE
1,–,O
2,Tanah,O
3,Air,O
4,kini,O
...,...,...
25427,watnita,O
25428,se,O
25429,setnita,O
25430,11apartemen-apartemen,O


In [ ]:
df.to_csv("/content/drive/MyDrive/nolimit/NER/data/tambahan.txt", sep="\t", index=False, header=False)

## load dataset fix

In [ ]:
# load data

train_dataset_path = '/content/drive/MyDrive/nolimit/NER/data/train_preprocess_new.txt'
valid_dataset_path = '/content/SEA_NLP_Workshop/indonlu/dataset/nergrit_ner-grit/valid_preprocess.txt'
test_dataset_path = '/content/SEA_NLP_Workshop/indonlu/dataset/nergrit_ner-grit/test_preprocess_masked_label.txt'

In [ ]:
train_dataset = NerGritDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = NerGritDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = NerGritDataset(test_dataset_path, tokenizer, lowercase=True)

# train_loader = NerDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=64, num_workers=2, shuffle=True)
# valid_loader = NerDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=64, num_workers=2, shuffle=False)
# test_loader = NerDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=64, num_workers=2, shuffle=False)

train_loader = NerDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=64, shuffle=True)
valid_loader = NerDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=64, shuffle=False)
test_loader = NerDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=64, shuffle=False)

In [ ]:
print(train_dataset[1000])

(array([    2,  6369, 13224,     7, 26203, 30464, 30465,   407,   596,
       14472,     7, 30468, 26099, 30468, 28534,    37, 30468,    41,
       16754,  8023,  2002,  2158,  1751,  2747, 30470,     3]), array([-1,  0,  1,  1,  2,  3,  4,  5,  6,  7,  7,  8,  9, 10, 11, 11, 12,
       13, 14, 15, 16, 17, 18, 19, 20, -1]), array([6, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5]), ['John', 'Towner', 'Williams', '(', ')', 'merupakan', 'seorang', 'komposer', ',', 'konduktor', ',', 'pianis', ',', 'dan', 'produser', 'rekaman', 'musik', 'asal', 'Amerika', 'Serikat', '.'])


In [ ]:
w2i, i2w = NerGritDataset.LABEL2INDEX, NerGritDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'I-PERSON': 0, 'B-ORGANISATION': 1, 'I-ORGANISATION': 2, 'B-PLACE': 3, 'I-PLACE': 4, 'O': 5, 'B-PERSON': 6}
{0: 'I-PERSON', 1: 'B-ORGANISATION', 2: 'I-ORGANISATION', 3: 'B-PLACE', 4: 'I-PLACE', 5: 'O', 6: 'B-PERSON'}


In [ ]:
def word_subword_tokenize(sentence, tokenizer):
    # Add CLS token
    subwords = [tokenizer.cls_token_id]
    subword_to_word_indices = [-1] # For CLS

    # Add subwords
    for word_idx, word in enumerate(sentence):
        subword_list = tokenizer.encode(word, add_special_tokens=False)
        subword_to_word_indices += [word_idx for i in range(len(subword_list))]
        subwords += subword_list

    # Add last SEP token
    subwords += [tokenizer.sep_token_id]
    subword_to_word_indices += [-1]

    return subwords, subword_to_word_indices

## model finetunning

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)
model = model.cpu()

In [ ]:
# Train
n_epochs = 1
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)

    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cpu')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = ner_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)

    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cpu')

        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = ner_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))

    metrics = ner_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

(Epoch 1) TRAIN LOSS:1.9335 LR:0.00002000:  15%|█▍        | 4/27 [03:16<18:49, 49.09s/it]


KeyboardInterrupt: ignored

In [ ]:
# save the model to disk

import pickle

filename = '/content/drive/MyDrive/nolimit/NER/model/model_indobert_combinedata_1ep.pkl'
pickle.dump(model, open(filename, 'wb'))

## load model

In [ ]:
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

from nltk.util import ngrams

from nltk.corpus import stopwords
sw = stopwords.words("indonesian")

def text_cleaning(text, remove_stop_words):
    # Clean the text

    text = text.lower() #lowercase atau case folding
    text = re.sub('@[^\s]+', ' ', text) #remove username
    text = re.sub('\[.*?\]', ' ', text) # remove square brackets
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # remove URLs
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) # remove punctuation
    # text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub("[!@#$%^&*()[]{};:,./<>?\|`~-=_+–]", ' ', text)
    text = re.sub("–", ' ', text)

    # Optionally, remove stop words
    if remove_stop_words:
        # load stopwords
        text = text.split()
        text = [w for w in text if not w in sw] #6
        text = " ".join(text)
        text = text.lower()

    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
from tqdm import tqdm

from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pickle

model = pickle.load(open('/content/drive/MyDrive/nolimit/NER/model/model_indobert_new_DO_4_combinedata.pkl', 'rb'))

In [ ]:
model

BertForWordClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
def word_subword_tokenize(sentence, tokenizer):
    # Add CLS token
    subwords = [tokenizer.cls_token_id]
    subword_to_word_indices = [-1] # For CLS

    # Add subwords
    for word_idx, word in enumerate(sentence):
        subword_list = tokenizer.encode(word, add_special_tokens=False)
        subword_to_word_indices += [word_idx for i in range(len(subword_list))]
        subwords += subword_list

    # Add last SEP token
    subwords += [tokenizer.sep_token_id]
    subword_to_word_indices += [-1]

    return subwords, subword_to_word_indices

In [ ]:
text = word_tokenize('muhammad fahmi pergi ke mall taman anggrek membeli kue bantal')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,muhammad,B-PERSON
1,fahmi,I-PERSON
2,pergi,O
3,ke,O
4,mall,B-PLACE
5,taman,I-PLACE
6,anggrek,I-PLACE
7,membeli,O
8,kue,O
9,bantal,O


In [ ]:
text = word_tokenize('Joko Widodo adalah presiden ketujuh Republik Indonesia')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,Joko,B-PERSON
1,Widodo,I-PERSON
2,adalah,O
3,presiden,O
4,ketujuh,O
5,Republik,B-PLACE
6,Indonesia,B-PLACE


In [ ]:
# Evaluate on test
# model.eval()
# torch.set_grad_enabled(False)

# total_loss, total_correct, total_labels = 0, 0, 0
# list_hyp, list_label = [], []

# pbar = tqdm(test_loader, leave=True, total=len(test_loader))
# for i, batch_data in enumerate(pbar):
#     _, batch_hyp, _ = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cpu')
#     list_hyp += batch_hyp

# # Save prediction
# df = pd.DataFrame({'label':list_hyp}).reset_index()
# df.to_csv('pred.txt', index=False)

# print(df)

In [ ]:
text = word_tokenize('Bung Tomo adalah pahlawan nasional Republik Indonesia')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,Bung,B-PERSON
1,Tomo,I-PERSON
2,adalah,O
3,pahlawan,O
4,nasional,O
5,Republik,O
6,Indonesia,B-PLACE


In [ ]:
text = word_tokenize('Nama saya Budi. Saya lahir di tahun 1981. Sekarang saya tinggal di Ubud, Bali.')
#My name is Budi. I was born in 1981. I am currently living in Ubud, Bali.
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})


,words,label
0,Nama,O
1,saya,O
2,Budi,B-PERSON
3,.,O
4,Saya,O
5,lahir,O
6,di,O
7,tahun,O
8,1981,O
9,.,O


In [ ]:
text = word_tokenize('bank bca')
#Budi has arrived in front of the BCA Tower.
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})


,words,label
0,bank,B-ORGANISATION
1,bca,I-ORGANISATION


In [ ]:
text = word_tokenize('Saya sudah sampai di depan menara bca')
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
logits = model(subwords, subword_to_word_indices)[0]

preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
labels = [i2w[preds[i]] for i in range(len(preds))]

pd.DataFrame({'words': text, 'label': labels})

,words,label
0,Saya,O
1,sudah,O
2,sampai,O
3,di,O
4,depan,O
5,menara,B-PLACE
6,bca,I-PLACE


## Predict combine Label B & I dan generate statement

prepros spasi

SNA ONM, NER & Statement => awal Q2

In [ ]:
# text_mentah = """ Sampai 31 Oktober 2022 jumlah kasus (gagal ginjal akut) ada 304,
# dirawat sebanyak 46 kasus, meninggal 159 kasus atau 52 persen,
# dan yang sembuh 99 orang, kata Juru Bicara Kemenkes Mohammad Syahril
# dalam konferensi pers daring di Jakarta, Selasa (1/11/2022) """

# text_mentah = 'KOMPAS.com – Pelatih Belanda, Louis van Gaal, mempunyai cara unik untuk mempersiakan diri melawan penyerang lincah, Argentina, Lionel Messi, di 8 besar Piala Dunia 2022. Belanda akan melakoni pertandingan melawan Argentina dalam babak perempat final Piala Dunia 2022. Pertandingan Belanda vs Argentina dalam jadwal Piala Dunia 2022 berlangsung di Stadion Lusail, Doha, Qatar, pada Sabtu (10/12/2022). Belanda bakal mewaspadai pergerakan Lionel Messi saat menghadapi Argentina pada 8 besar Piala Dunia 2022. Louis van Gaal sudah mempunyai cara khusus agar pemain-pemain Belanda terbiasa untuk berhadapan dengan Lionel Messi di lapangan. Baca juga: Prediksi Skor dan Line Up Belanda Vs Argentina di 8 Besar Piala Dunia 2022 Mantan pelatih Manchester United itu menyuruh Noa Lang untuk meniru gaya bermain Messi dalam sesi latihan Belanda. Noa Lang sejauh ini belum pernah tampil bersama Belanda. Ia hanya menjadi penghangat bangku cadangan selama empat laga Piala Dunia 2022. “Dia (Noa Lang) juga berpikir bahwa masuk akal bagi dia untuk menirukan Lionel Messi. Itu hebat,” kata Louis van Gaal seraya tertawa dalam sesi konferensi pers jelang duel Belanda vs Argentina, dikutip dari Voetbal International. Secara profil, Noa Lang yang berpostur 176 cm, barangkali menjadi pemain Belanda yang paling menyerupai Messi (170 cm). Pemain Club Brugge itu juga menonjol dalam aspek dribel. Di lain sisi, dua pemain Belanda, Frenkie de Jong dan Nathan Ake, sempat mengungkapkan kesulitan tim mencari cara menghentikan Messi.'

# text_mentah = 'muhammad fahmi pamungkas dan adam santoso pergi ke mall taman anggrek membeli kue bantal di toko teman mereka yang bernama rizki yang punya palang merah indonesia'

# text_mentah = """ JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng. Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang. "Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai. "Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya. Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada. Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi. "Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan. Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung. Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully. Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara. "Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan. "Bahkan (Megawati) sampai (mengatakan), nggak papa saya di-bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya. Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023)."""

text_mentah = """JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng. Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang. "Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023). Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai. "Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya. Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada. Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi. "Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan. Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung. Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully. Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara. "Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan. "Bahkan (Megawati) sampai (mengatakan), nggak papa saya di-bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya. Lagi pula, lanjut Puan, dalam pidatonya Megawati juga sempat memuji Jokowi dengan menyebut bahwa mantan Gubernur DKI Jakarta itu pintar sehingga PDI-P bersedia mengusungnya jadi pemimpin. Oleh karenanya, Puan menegaskan bahwa Megawati tak bermaksud buruk, apalagi menghina Jokowi lewat pidatonya. "Jadi bukan ada yang rasa yang mau merendahkan atau sepertinya nggak hormat, itu justru kebalikan," tutur mantan Menteri Koordinator Bidang Pembangunan Manusia dan Kebudayaan (Menko PMK) itu. Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023). Di hadapan ribuan kader yang hadir, Megawati berkelakar, nasib Jokowi tidak akan sama seperti sekarang jika tak ada PDI-P. "Pak Jokowi itu kayak begitu lho, mentang-mentang. Lah iya, padahal Pak Jokowi kalau enggak ada PDI Perjuangan juga, aduh, kasihan dah," kata Megawati sambil tertawa. Seloroh Megawati tersebut disambut tawa para tamu undangan, termasuk Jokowi yang duduk di barisan kursi tamu terdepan. Presiden kelima RI itu bilang, seandainya PDI-P tak memberikan dukungan, maka Jokowi tidak akan jadi presiden seperti sekarang. "Lho legal formal lho, beliau jadi presiden itu enggak ada kan ini, legal formal diikuti terus sama saya," ujarnya. Pernyataan Megawati itu pun memunculkan kritik dari sejumlah kalangan. Putri Soekarno itu bahkan dianggap merendahkan presiden. """

In [ ]:
df = pd.read_csv('/content/Politik_Indonesia_OnlineMedia_08-Jan-2023_15-Jan-2023_5PZf95F59l.csv', on_bad_lines='skip',
                 sep=';')

In [ ]:
df.head()[['body']]

In [ ]:
df.columns

In [ ]:
df['body'][4]

In [ ]:
text_mentah = df['body'][4]

In [ ]:
for i in range(len(hasil)):

  # --------------- PERSON -----------------------------
  try:
    if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]!='I-PERSON':
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'
      # if hasil['words'][i]==hasil['words'][i]:
      #   pass
    else:
      pass

  except:
    if hasil['label'][i]=='B-PERSON' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'
    else:
      pass

  # try:
  #   if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON' and hasil['label'][i+2]=='I-PERSON':
  #     print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PERSON')
  #     output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PERSON'
  #   elif hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON':
  #     print(hasil['words'][i],hasil['words'][i+1], '=> PERSON')
  #     output[hasil['words'][i],hasil['words'][i+1]] = 'PERSON'
  #   else:
  #     pass

  # except:
  #   pass

## **dibawah gunakan untuk predict ner**

In [ ]:
model = pickle.load(open('/content/drive/MyDrive/nolimit/NER/model/model_indobert_new_DO_5_combinedata.pkl', 'rb'))

In [ ]:
text_mentah = """JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.

Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.

"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).

Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.

"Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya.

Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada.

Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi.

"Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan.

Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung.

Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully.

Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara.

"Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan.

"Bahkan (Megawati) sampai (mengatakan), nggak papa saya di- bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya.

Lagi pula, lanjut Puan, dalam pidatonya Megawati juga sempat memuji Jokowi dengan menyebut bahwa mantan Gubernur DKI Jakarta itu pintar sehingga PDI-P bersedia mengusungnya jadi pemimpin.

Oleh karenanya, Puan menegaskan bahwa Megawati tak bermaksud buruk, apalagi menghina Jokowi lewat pidatonya.

"Jadi bukan ada yang rasa yang mau merendahkan atau sepertinya nggak hormat, itu justru kebalikan," tutur mantan Menteri Koordinator Bidang Pembangunan Manusia dan Kebudayaan (Menko PMK) itu.

Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023).

Di hadapan ribuan kader yang hadir, Megawati berkelakar, nasib Jokowi tidak akan sama seperti sekarang jika tak ada PDI-P.

"Pak Jokowi itu kayak begitu lho, mentang-mentang. Lah iya, padahal Pak Jokowi kalau enggak ada PDI Perjuangan juga, aduh, kasihan dah," kata Megawati sambil tertawa.

Seloroh Megawati tersebut disambut tawa para tamu undangan, termasuk Jokowi yang duduk di barisan kursi tamu terdepan.

Presiden kelima RI itu bilang, seandainya PDI-P tak memberikan dukungan, maka Jokowi tidak akan jadi presiden seperti sekarang.

"Lho legal formal lho, beliau jadi presiden itu enggak ada kan ini, legal formal diikuti terus sama saya," ujarnya.

Pernyataan Megawati itu pun memunculkan kritik dari sejumlah kalangan. Putri Soekarno itu bahkan dianggap merendahkan presiden."""

#### 1

In [ ]:
# 1

# text_mentah = """
# Jakarta: Menteri Keuangan (Menkeu) Sri Mulyani Indrawati dikabarkan tengah digadang menduduki kursi empuk Gubernur Bank Indonesia (BI). Bendahara Negara itu bersama Gubernur BI Perry Warjiyo, Ketua Dewan Komisioner Lembaga Penjamin Simpanan (LPS) Purbaya Yudhi Sadewa, dan Deputi Gubernur Senior BI Destry Damayanti disebut-sebut masuk ke dalam bursa calon Gubernur BI selanjutnya.

# Masa jabatan Perry Warjiyo sebagai Gubernur BI pada tahun ini akan habis pada Mei 2023. Dengan begitu, Presiden Joko Widodo akan mengajukan nama calon baru Gubernur BI kepada Dewan Perwakilan Rakyat (DPR) untuk dilakukan uji kepatutan dan kelayakan.

# Lalu, berapakah sebenarnya gaji Gubernur Bank Indonesia sampai-sampai Sri Mulyani kepincut buat duduk jadi orang nomor satu di lembaga negara yang independen tersebut?

# Bagaimana tanggapan anda mengenai artikel ini?

# Gaji Gubernur Bank Indonesia

# Tidak ada satu data resmi dan pasti mengenai gaji Gubernur Bank Indonesia. Namun, berdasarkan hasil penelusuran yang dirangkum Medcom.id dari berbagai sumber, besaran gaji Gubernur Bank Indonesia mencapai lebih dari Rp100 juta per bulan.

# Beberapa sumber bahkan menyebutkan besaran gaji Gubernur Bank Indonesia berada di kisaran Rp170 juta hingga Rp200 juta per bulannya.

# Fantastis memang, namun jumlah tersebut sebanding dengan beratnya tugas yang diemban Gubernur Bank Indonesia. Diantaranya menetapkan dan melaksanakan kebijakan moneter secara berkelanjutan, konsisten, dan transparan.

# Selanjutnya, mengatur dan menjaga kelancaran sistem pembayaran. Serta, menetapkan dan melaksanakan kebijakan makroprudensial.

# Gaji menteri

# Sementara itu, gaji menteri diatur dalam Peraturan Pemerintah Republik Indonesia Nomor 75 Tahun 2000. Dalam beleid itu disebutkan bahwa gaji seorang menteri di Indonesia sebesar Rp5,04 juta per bulan.

# Eits, tunggu dulu. Jumlah uang yang diterima seorang menteri dalam sebulannya bukan cuma gaji tok. Ada juga tunjangan jabatan menteri sebesar Rp13,608 juta per bulan. Artinya, seorang menteri dalam sebulannya mendapatkan total sebanyak Rp18,648 juta.

# Selain tunjangan kinerja, menteri negara juga mendapatkan fasilitas dan tunjangan lain untuk jabatan tertinggi pejabat negara. Dalam Peraturan Menteri Keuangan (PMK) Nomor 176/PMK.02/2015, menteri juga mendapatkan kendaraan dinas, rumah jabatan, dan jaminan kesehatan.

# Gimana sobat Medcom? Bikin ngiler ya gaji seorang menteri dan Gubernur Bank Indonesia.
# """

In [ ]:
text_mentah

'JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.\n\nMenurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.\n\n"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).\n\nPuan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.\n\n"Kenapa di acara PDI Perjuangan bicara sepe

#### 2

In [ ]:
# 2

# text_mentah = """JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.

# Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.

# "Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).

# Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.

# "Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya.

# Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada.

# Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi.

# "Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan.

# Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung.

# Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully.

# Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara.

# "Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan.

# "Bahkan (Megawati) sampai (mengatakan), nggak papa saya di- bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya.

# Lagi pula, lanjut Puan, dalam pidatonya Megawati juga sempat memuji Jokowi dengan menyebut bahwa mantan Gubernur DKI Jakarta itu pintar sehingga PDI-P bersedia mengusungnya jadi pemimpin.

# Oleh karenanya, Puan menegaskan bahwa Megawati tak bermaksud buruk, apalagi menghina Jokowi lewat pidatonya.

# "Jadi bukan ada yang rasa yang mau merendahkan atau sepertinya nggak hormat, itu justru kebalikan," tutur mantan Menteri Koordinator Bidang Pembangunan Manusia dan Kebudayaan (Menko PMK) itu.

# Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023).

# Di hadapan ribuan kader yang hadir, Megawati berkelakar, nasib Jokowi tidak akan sama seperti sekarang jika tak ada PDI-P.

# "Pak Jokowi itu kayak begitu lho, mentang-mentang. Lah iya, padahal Pak Jokowi kalau enggak ada PDI Perjuangan juga, aduh, kasihan dah," kata Megawati sambil tertawa.

# Seloroh Megawati tersebut disambut tawa para tamu undangan, termasuk Jokowi yang duduk di barisan kursi tamu terdepan.

# Presiden kelima RI itu bilang, seandainya PDI-P tak memberikan dukungan, maka Jokowi tidak akan jadi presiden seperti sekarang.

# "Lho legal formal lho, beliau jadi presiden itu enggak ada kan ini, legal formal diikuti terus sama saya," ujarnya.

# Pernyataan Megawati itu pun memunculkan kritik dari sejumlah kalangan. Putri Soekarno itu bahkan dianggap merendahkan presiden."""

#### 3

In [ ]:
# 32

# text_mentah = """REPUBLIKA.CO.ID, JAKARTA -- Presiden Joko Widodo (Jokowi) menyebut akan kembali menghentikan ekspor bahan mineral pada tahun ini. Setelah nikel dan bauksit, Jokowi akan menghentikan ekspor tembaga pada tahun ini.

# "Saya sudah sampaikan lagi bauksit di Desember kemarin bauksit stop bulan Juni. Nanti sekali lagi mau saya umumkan lagi tembaga stop tahun ini. Stop," kata Jokowi di Mandiri Investment Forum, Jakarta, Rabu (1/2/2023).

# Jokowi menjelaskan, keputusannya untuk menghentikan ekspor tembaga tersebut mempertimbangkan pembangunan smelter milik PT Freeport Indonesia di NTB yang sudah lebih dari 51 persen jadi. Ia pun kembali menegaskan, mayoritas saham PT Freeport saat ini sudah menjadi milik Indonesia.

# "Karena saya cek kemarin smelternya Freeport dan smelter yang ada di NTB sudah lebih dari 50 persen jadi. Freeport itu sudah lebih 51 persen jadi. Jadi berani kita stop, dan supaya ingat Freeport itu sudah mayoritas milik kita. Jadi jangan terbayang-bayang lagi Freeport itu masih miliknya Amerika. Sudah mayoritas kita miliki," jelas Jokowi.

# Menurutnya, larangan ekspor bahan mineral mentah ini bisa memberikan nilai tambah kepada negara. Ia mencontohkan larangan bauksit. Selama ini, ekspor bahan mentah bauksit Indonesia masuk dalam tiga besar dunia. Namun ekspor hasil produksi barang hilirisasi justru masih sedikit.

# "Tapi ekspor alumunium kita nomor 33. Mentahnya nomor tiga kok. Barang setengah jadi barang jadinya di 33. Apalagi ekspor panel surya kita nomor 31. Padahal bahannya ada di sini dan kalau dikerjakan panel surya itu nilai tambahnya sampai 194 kali," ujarnya.

# Jokowi mengatakan, selama ini Indonesia terlalu nyaman untuk mengekspor bahan mentah mineral. Ia pun meminta agar Indonesia bisa mencontoh Cina.

# "RRT China ekspornya nomor 18. Tapi ekspor panel suryanya nomor 1 di dunia. Terus barangnya ini dari mana? Barang mentahnya dari mana? 80 persen lebih dari kita," kata dia.

# Untuk meningkatkan nilai tambah dari hilirisasi, Jokowi menekankan agar upaya ini konsisten terus dilakukan. Meskipun menghadapi gugatan, ia menegaskan agar para menterinya tak mundur melanjutkan proses hilirisasi yang sudah dibangun.

# "Ya ini semuanya harus konsisten dan harus dikawal. Kalau tidak kita mundur lagi ke belakang. Karena ini memang konsistensi dan... karena kalau engga kita balik lagi ke ekspor bahan mentah dan ga dapet nilai tambah sampai kapanpun," jelasnya.

# Selain memberikan nilai tambah, hilirasi minerba dan migas juga bisa membuka lapangan kerja baru di Indonesia. Upaya ini dinilainya akan memberikan lompatan bagi Indonesia untuk menjadi negara maju.

# "Proyeksi dampak hilirisasi minerba dan migas itu akan menambah PDB kita sebesar 699 dolar AS dan lapangan kerja yang akan terbuka di angka 8,8 juta. Ini sebuah dampak yang sangat besar sekali. Membuka lapangan kerja yang sebesar-besarnya," jelasnya."""

#### 4

In [ ]:
# # 4

# text_mentah = """
# Jakarta, CNN Indonesia --

# Badai pemutusan hubungan kerja (PHK) di e-commerce masih berlanjut imbas ekonomi lesu. Tak hanya PHK, e-commerce JD.ID bahkan harus gulung tikar di awal tahun ini.

# JD.ID merupakan perusahaan patungan e-commerce China JD.com dan Provident Capital. Marketplace ini resmi menutup layanan belanja online mereka per 31 Maret 2023, dan melayani pesanan terakhir pada 15 Februari 2023.

# Sebelum mengumumkan akan menutup layanannya, JD.ID menutup cabang logistiknya JDL Express Indonesia per 22 Januari 2023. Perusahaan juga melakukan PHK secara bertahap.

# Pada Desember 2022, JD.ID melakukan PHK kepada 30 persen atau 200 karyawannya. Langkah PHK ini bukan yang pertama. Perusahaan juga mengambil langkah serupa pada Mei di tahun yang sama.

# Selain JD.ID, PT Shopee Indonesia juga memangkas jumlah karyawannya pada September tahun lalu.

# Head of Public Affairs Shopee Indonesia Radynal Nataprawira menjelaskan PHK merupakan langkah terakhir yang harus ditempuh perusahaannya sebagai efisiensi, setelah sebelumnya melakukan penyesuaian melalui beberapa perubahan kebijakan bisnis.

# Senada, PHK juga dilakukan PT Goto Gojek Tokopedia Tbk atau GoTo terhadap 12 persen dari total karyawannya atau sebanyak 1.300 orang.

# PHK dilakukan agar perusahaan lebih agile atau lincah dan bisa menjaga tingkat pertumbuhan, sehingga terus memberikan dampak positif bagi jutaan konsumen, mitra pengemudi, dan pedagang.

# Menanggapi gempuran PHK yang terus terjadi di perusahaan e-commerce, Peneliti Center of Economic and Law Studies (Celios) Muhammad Andri Perdana mengatakan pendanaan yang seret menjadi alasan kuat mengapa para e-commerce goyah.

# Andri menegaskan investor sudah semakin selektif dan kini beralih ke instrumen investasi yang lebih aman ketimbang harus menyuntikkan dana ke e-commerce. Dengan alasan itu, Andri melihat promo e-commerce saat ini sangat terbatas.

# "Di saat pertumbuhan e-commerce meredup, sedangkan return obligasi negara, seperti US Treasury meningkat padahal tingkat resikonya jauh lebih rendah, maka sekarang e-commerce kesulitan mendapatkan dana segar dari investor. Mau mendapatkan pendanaan dari bank pun juga mahal karena suku bunga ikut meningkat," jelasnya kepada CNNIndonesia.com, Rabu (1/2).

# Selain masalah pendanaan, minimnya promo dan persaingan antar e-commerce juga menjadi penyebab runtuhnya satu per satu bisnis belanja online tersebut.

# Andri menilai meningkatnya mobilitas masyarakat setelah pencabutan pemberlakuan pembatasan kegiatan masyarakat (PPKM) di seluruh wilayah Indonesia pada akhir Desember 2022.

# Menurutnya, banyak konsumen yang akhirnya kembali berbelanja di toko konvensional ketimbang check out di e-commerce. Apalagi, semenjak suntikan dana terhadap e-commerce seret, para marketplace tersebut kini minim promo.

# Ia menilai daya beli masyarakat juga belum pulih sepenuhnya karena inflasi tidak dibarengi kenaikan pendapatan rumah tangga.

# Kondisi ini pada akhirnya membuat masyarakat lebih memilih pilihan yang lebih ekonomis dengan belanja di toko konvensional, meninggalkan e-commerce. Terlebih, Andri menilai sekarang e-commerce semakin dituntut untuk mengurangi kerugian.

# "Sehingga sekarang banyak e-commerce menambah biaya admin dan potongan penjualan kepada penjual. Biaya ini biasanya oleh penjual akan dibebankan ke pembeli yang membuat harga-harga di e-commerce semakin tidak ekonomis bagi pembeli," jelasnya.

# Terkait persaingan e-commerce yang semakin ketat. Ia menilai hadirnya social commerce seperti TikTok Shop yang viral belakangan ini dengan istilah "keranjang kuning", juga menjadi pesaing marketplace.

# Ia menjelaskan pasar e-commerce Indonesia banyak beririsan dengan platform media sosial. Namun, platform media sosial seperti Facebook hingga TikTok terus berinovasi menghadirkan layanan belanja online milik mereka masing-masing.

# Oleh karena itu, Andri menyarankan e-commerce mengambil langkah tepat dalam menjaga pasar. Tujuannya agar para pengguna tetap menggunakan platform belanja online mereka.

# "Seperti memberikan insentif bagi penyebar link produk di sosial media atau bahkan memperkuat positioning mereka pada segmen-segmen yang menjadi target sehingga pengguna sosial media tetap mengandalkan e-commerce tersebut untuk membeli barang," tandasnya.

# Di lain sisi, Direktur Indonesia Development and Islamic Studies (IDEAS) Yusuf Wibisono menyoroti secara khusus soal badai PHK di industri startup, termasuk e-commerce. Ia memproyeksi PHK di industri digital masih berpeluang besar berlanjut di tahun ini.

# Bagi perusahaan mapan, Yusuf menilai kemunduran usaha mereka banyak disebabkan aksi over ekspansi di masa pandemi. Memang permintaan terhadap produk dan jasa digital melonjak drastis pada saat itu, sehingga banyak perusahaan digital merekrut tenaga kerja secara besar-besaran.

# Sayang, seiring dengan penghapusan berbagai pembatasan sosial selepas pandemi membuat permintaan terhadap produk dan jasa digital menurun. Pada akhirnya PHK e-commerce tidak bisa dihindari.

# Sementara itu, Yusuf merinci tiga faktor PHK di startup digital yang banyak mengandalkan pembiayaan non-bank. Pertama, model bisnis yang tidak solid. Kedua, persaingan yang semakin keras. Ketiga, dukungan pendanaan yang melemah seiring pengetatan moneter dunia dan berakhirnya era suku bunga rendah.

# Ia juga menyinggung soal fenomena 'bakar uang' di dunia startup. Menurutnya, strategi tersebut jika tidak kunjung menghasilkan revenue dan profit memadai hanya akan berujung terhentinya kucuran dana dari investor.

# "Akibatnya startup banyak yang mulai berhenti melakukan strategi 'bakar uang' untuk akuisisi konsumen, melakukan efisiensi seperti PHK karyawan, bahkan tidak sedikit yang mengalami kebangkrutan," jelasnya.

# Yusuf menegaskan bahwa persaingan memperebutkan pasar semakin keras dan akuisisi konsumen tidak lagi bisa hanya mengandalkan promo dan diskon. Sementara penyedia dana tidak lagi memiliki keleluasaan seiring kenaikan suku bunga dan likuiditas yang semakin ketat.
# """

#### 5

In [ ]:
# # 5

# text_mentah = """
# Medan, CNN Indonesia --

# Gubernur Jawa Barat yang juga politikus Golkar Ridwan Kamil membagikan sepeda kepada warga saat berkunjung ke Medan, Sumatera Utara, Rabu (1/2).

# Momen itu terjadi saat sosialisasi kredit bjb Mesra (Masyarakat Ekonomi Sejahtera) Pemprov Jawa Barat. Sosialisasi dilakukan di Lapangan Benteng Medan.

# Momen bagi-bagi sepeda diawali dengan pertanyaan yang dilontarkan Kang Emil.

# "Sebutkan nama lima kota di Jawa Barat," kata Kang Emil.

# Beberapa warga langsung berebut mengangkat tangan untuk menjawab. Mereka yang dipilih lalu maju ke depan dan menjawab pertanyaan dari Kang Emil.

# Mereka yang menjawab dengan benar diberikan sepeda oleh Kang Emil. Mirip cara Presiden Jokowi di berbagai momen saat bertemu warga.

# "Jadi sabar, pasti dapat, tapi porsi untuk ibu ibu lebih banyak," ungkapnya disambut tepuk tangan meriah.

# Dalam kesempatan itu, Kang Emil sempat berinteraksi dengan warga Medan. Pidatonya pun diselingi dengan guyonan sehingga memancing gelak tawa.

# Dia mengatakan kini pedagang kecil tak lagi terjerat rentenir jika mengajukan kredit bjb Mesra. Kondisi tersebut juga membuat kepala daerah senang karena warganya terbantu.

# "Kalau rakyat senang pemimpin juga senang, jadi memang satu hati dengan rakyat," ucapnya.

# Ridwan Kamil menyatakan kredit bjb Mesra merupakan program bersama antara Pemerintah Provinsi Jawa Barat bersama bjb.

# Tujuannya membantu masyarakat agar terhindar pinjol (pinjaman online) dan terhindar dari jeratan rentenir.

# Menurutnya. banyak ibu rumah tangga di Jawa Barat yang sudah terbebas dari rentenir dengan memanfaatkan Kredit bjb Mesra. Kredit ini merupakan kredit tanpa agunan dengan nilai kredit Rp 500 ribu sampai Rp 5 juta.
# """

#### 6

In [ ]:
# # 6

# text_mentah = """
# Jakarta, CNN Indonesia --

# Anggota Komisi VII DPR RI dari Fraksi PKS Mulyanto meminta agar Presiden Joko Widodo mencopot dan mengganti Kepala Badan Riset dan Inovasi Nasional (BRIN) Laksana Tri Handoko. Ia menilai Laksana gagal dalam mengkonsolidasikan baik lembaga, SDM, maupun anggaran BRIN.

# Mulyanto berpendapat selama kepemimpinan Laksana, muncul berbagai kejadian kurang baik terkait BRIN. Desakan Mulyanto sejalan dengan Komisi VII DPR yang mendesak pemerintah untuk segera mengganti Laksana. Hal itu merupakan kesimpulan rapat dengar pendapat dengan Kepala BRIN pada Senin (30/1) lalu.

# "Saya menganggap pimpinan BRIN yang ada sekarang ini tidak dapat mengkonsolidasikan lembaga-lembaga di bawah kewenangannya. Karena itu saya mengusulkan agar pimpinan BRIN sekarang diganti saja," kata Mulyanto dikutip dari situs resmi PKS, Rabu (1/2).

# Mulyanto selanjutnya menyebut kapasitas impelementasi program dalam tubuh BRIN sangat lemah dan tidak implementatif, sehingga muncul beberapa kasus terkait BRIN. Ia kemudian menyinggung salah satu kejadian yang sempat menghebohkan masyarakat yang menurutnya disebabkan tidak rapinya koordinasi di BRIN.

# Salah satu peneliti BRIN akhir tahun lalu sempat memprediksi akan ada potensi banjir besar yang akan melanda di wilayah Jabodetabek khususnya Tangerang, Banten akibat curah hujan ekstrem dan badai dahsyat pada 28 Desember 2022.

# Wilayah Banten yang diperkirakan akan terdampak di antaranya Cilegon, Serang, Pandeglang, Rangkasbitung, dan Tangerang. Prakiraan BRIN tersebut berdasarkan analisis data Satellite Early Warning System (Sadewa)

# Mulyanto lantas menyoroti prediksi itu sempat menimbulkan kehebohan masyarakat Banten. Menurutnya, ketika muncul indikasi awal akan terjadi badai besar, salah satu peneliti BRIN itu seharusnya melakukan koordinasi dan validasi data langsung sebelum menyampaikan pernyataan sehingga tidak membuat warga panik.

# "Apa kewenangannya? walaupun saya tahu BRIN melakukan study early warning system dengan bantuan Jerman. Data-data itu kuat. Tapi yang berhak menyampaikan ke publik itu BMKG," ujar Mulyanto.

# "Sekarang kita dikejutkan lagi, seorang periset memberikan segepok data APBN yang bersifat rahasia, detil, kepada wartawan. Itu apakah terkendali atau tidak dokumen seperti itu," imbuhnya.

# Lebih lanut, Mulyanto mengaku tak heran apabila Badan Pemeriksa Keuangan (BPK) menemukan persoalan anggaran infrastruktur 2022 di BRIN, pun dengan yang Ombudsman menemukan berbagai persoalan terkait SDM.

# "Jadi cita-cita ingin mengkonsolidasikan, mengintegrasikan lembaga riset tidak terjadi. Yang bisa dilakukan kepala BRIN saat ini hanya menggabungkan status kelembagaan saja. Di dalamnya konsolidasi anggaran, program, tidak jalan," tegas Mulyanto.

# "Anggaran BRIN yang kita harapkan menjadi Rp24 triliun, adanya kurang lebih hanya Rp6 sampai Rp7 triliun. Padahal semua lembaga sudah melebur," imbuhnya.

# Komisi VII DPR sebelumnya mendesak pemerintah untuk segera mengganti Kepala BRIN Laksana Tri Handoko. Desakan itu merupakan kesimpulan rapat dengar pendapat dengan Kepala BRIN yang dipimpin Ketua Komisi VII DPR Sugeng Suparwoto dari Fraksi Partai NasDem pada Senin (30/1).

# Komisi VII DPR Juga merekomendasikan agar dilakukan audit khusus terhadap penggunaan anggaran BRIN tahun 2022 oleh BPK. Sementara itu, Laksana Tri Handoko enggan menanggapi desakan mundur dari Komisi VII DPR.

# "Saya enggak ada tanggapan kalau mengenai itu, entar aja," ujar Handoko di kantor BRIN Thamrin, Jakarta Pusat, Senin (31/1).

# Keterangan tersebut diberikan Handoko usai dirinya merampungkan rapat di kantornya pada pukul 18.50 WIB. Handoko saat itu juga tidak menjelaskan rapat apa yang tengah dibahas olehnya.
# """

#### 7

In [ ]:
# # 7

# text_mentah = """
# Harianjogja.com, JOGJA —Sebanyak 85 orang karyawan Trans Jogja yang bertugas diselter terkena pemutusan hubungan kerja (PHK) sejak awal 2023. Mereka mengadu ke DPRD DIY dengan harapan dapat dipekerjakan kembali, Rabu (1/2/2023).

# Koordinator Paguyuban Karyawan Selter Trans Jogja Yandi Rustanto menjelaskan sebanyak 85 pekerja Trans Jogja diberhentikan secara sepihak tanpa ada pemberitahuan dan surat resmi. PHK itu diberlakukan sejak 1 Januari 2023 lalu dengan secara tiba-tiba tidak diberikan jadwal oleh pihak Trans Jogja. Mereka adalah pekerja outsourcing yang sebagian besar telah bekerja antara 10 sampai 15 tahun dengan usia layak kerja.

# : PHK Buruh, Perusahaan Operator Bus Trans Jogja Digugat

# “Kami tahunya kalau di-PHK itu di jadwal kerja tidak keluar kalau biasanya kan ada jadwal siapa saja yang jaga, ini tidak muncul nama dan tanpa pemberitahuan sebelumnya. Menurut kami tanpa tranparansi,” katanya kepada wartawan saat beraudiensi di DPRD DIY, Rabu (1/2/2023).

# Kelompoknya sudah memberikan surat kepada Gubernur DIY dengan tembusan ke DPRD DIY atas PHK sepihak yang dilakukan Trans Jogja. Ia berharap melalui komunikasi dengan DPRD DIY puluhan orang yang rata-rata bertugas sebagai penjaga selter itu bisa dipekerjakana kembali. Mengingat mereka adalah tulang punggung untuk mencukupi kebutuhan keluarga.

# “Biasanya setiap akhir tahun diperpanjang tetapi kalau tidak diperpanjang itu diberitahu misalnya karena performa kerjanya tidak bagus atau dari usia sudah tidak memenuhi. Ini tidak ada pemberitahuan sama sekali, tidak dikaruhke sama sekali,” katanya.

# Ketua Komisi C DPRD DIY Gimmy Rusdin Sinaga yang menerima para mantan karyawan Trans Jogja akan mengupayakan mediasi dengan pihak PT Anindya Mitra International selaku pengelola Trans Jogja. “Karena keputusan sepenuhnya berada di Pemda DIY dan PT AMI tentu kami akan mencoba untuk mengkomunikasikan sesuai keinginan mereka, agar kalau bisa dipekerjakan kembali,” ujarnya.

# Direktur PT AMI Dyah Puspitasari saat dimintai konfirmasi menegaskan pemberitahuan tentang pemberhentian petugas selter itu telah dilakukan jauh hari sebelumnya. Langkah itu ditempuh bersama Dinas Perhubungan DIY karena keterbatasan anggaran operasional untuk petugas selter. Menurutnya pengurangan karyawan itu dilakukan telah sesuai prosedur sesuai ketentuan berlaku termasuk batas usia 40 tahun.

# “Selain itu sebenarnya kami masih memberikan kesempatan untuk bekerja kembali tetapi sebagai pramugara yang di dalam bus, bukan di selter,” katanya.
# """

#### 8

In [ ]:
# # 8

# text_mentah = """
# RADARDEPOK.COM - Rumah Sakit Umum Alia Hospital siap memberikan vaksinasi Covid-19 dosis keempat atau booster kedua untuk masyarakat.

# Lokasinya di Alia Hospital Jakarta Timur dan Alia Hospital Depok. "Tentunya, kami mendukung program Pemerintah dalam
# bidang kesehatan salah satunya vaksinasi Covid-19. Terkait target, tentunya kami ingin sebanyak-banyaknya sesuai dengan kuota dari Pemerintah," ujar Managing Director Alia
# Hospital, Bina Ratna, Kamis (2/2).

# Bina Ratna mengatakan, Alia Hospital berawal dari RSIA Bunda Aliyah Jakarta Timur yang berdiri tahun 2008. Memiliki layanan unggulan Neonatus Care Unit (NICU) dan High Risk Maternity Center (Fetomaternal).

# Sehingga, menjadi salah satu rumah sakit rujukan dan menjadi satelit RSCM. Menurutnya, di tahun 2018 mendirikan RSIA Bunda Aliyah Depok sebagai rumah sakit kedua. Dikatakannya, saat didirikan memiliki jumlah tempat tidur NICU terbesar di Kota Depok.

# Saat ini, seluruh rumah sakit tersebut telah berubah status menjadi Rumah Sakit Umum dengan penambahan jumlah kamar perawatan dan kelengkapan pusat unggulan yang disediakan. Yakni Kidney Center, Digestive Center, Cataract Center dan lainnya.

# "Jadi kami tidak hanya menyediakan layanan ibu dan anak, pelayanan medis untuk semuanya termasuk laki-laki. Terlebih saat ini kedua rumah sakit kami, telah mendapatkan Akreditasi dari KARS dengan tingkat Paripurna atas mutu dan layanannya," beber dia.

# Menteri Kesehatan, Budi Gunadi Sadikin pada acara Anniversary ke-15 Alia Hospital mengapresiasi serta mengucapkan selamat kepada Alia Hospital karena telah melakukan
# pengembangan rumah sakitnya. Terutama, soal layanan High Risk Maternity dan NICU -nya, layanan khusus bagi bayi yang lahir premature.

# Menurutnya, tercatat setiap 25 ribu setiap tahun bayi meninggal dunia disebabkan kelahiran premature dan 25 % penyakit jantung bawaan. "Pada waktu mendatang terkait pengurangan angka kematian bayi premature Kemenkes memiliki program," katanya.
# """

#### 9

In [ ]:
# # 9

# text_mentah = """
# ASKARA – Badan Akuntabilitas Publik Dewan Perwakilan Daerah membahas pentingnya implementasi integrasi data secara real time dalam persoalan Kepesertaan Badan Penyelenggara Jaminan Sosial/ Jaminan Kesehatan Nasional (BPJS/JKN) dengan Direktur BPJS/JKN, Kemendagri, dan Kemensos.

# Pada kesempatan itu, Dirjen Dukcapil Kemendagri Prof. Dr. Zudan Arif Fakrulloh, S.H. menyampaikan upaya Kemendagri dalam mendukung masalah penganggaran dan mendorong agar pemda dapat menyelesaikannya.

# “Terkait data, Kemendagri juga mendukung penyediaan data Data Terpadu Kesejahteraan Sosial (DTKS), dan BPJS kami harapkan bisa segera melakukan update secara real time,” ujar Dirjen Zudan di Gedung DPD RI Kompleks Parlemen Senayan Jakarta, Rabu (1/2).

# Kemendagri, lanjutnya, mendorong pelayanan proaktif jemput bola, agar segera terlaporkan jika ada perubahan. Kami juga mengharapkan penduduk yang datanya berubah untuk aktif melakukan perubahan data kependudukan.

# “Karena kami sifatnya menerima laporan dan merubah data jika ada permintaan meski sekarang kami sudah jemput bola," tuturnya.

# Ketua BAP DPD RI Ajiep Padindang menyatakan, BAP DPD RI menemukan beberapa permasalahan dalam implementasi BPJS/JKN sekarang ini, antara lain belum sinkronnya data antara dinas sosial dengan BPJS Kesehatan, pemutakhiran data base dan BPJS yang menjadi beban terhadap APBD, serta proses pelayanan BPJS.

# "Data base kepesertaan program JKN seharusnya diintegrasikan dengan data milik kementerian sosial, kemendagri, secara real time dan valid sehingga kemanfaatannya dapat dirasakan baik oleh daerah dan masyarakat," ujar Ajiep Padindang bersama Wakil Ketua BAP DPD RI Bambang Sutrisno, Mirati Dewaningsih, dan Arniza Nilawati pada rapat tersebut.

# Lebih lanjut Ajiep menerangkan meskipun jumlah kepesertaannya tinggi, namun BPJS Kesehatan sebagai pengelola program JKN-KIS pernah mengalami defisit pembiayaan sebesar 7,95 triliun rupiah pada tahun 2018, defisit 51 triliun rupiah pada tahun 2019, dan defisit sebesar 5,69 triliun rupiah pada tahun 2020 yang dapat mengancam keberlanjutan program.

# Pada tanggal 1 Januari 2021, pemerintah telah menaikkan iuran JKN-KIS pada semua kelas tetapi upaya ini masih belum dapat mengatasi persoalan defisit dengan optimal.

# "BAP DPD RI juga mendorong adanya sinergi dan sinkronisasi data yang kuat antara DPD RI dengan Kementerian Sosial, Kementerian Dalam Negeri dan Kementerian Kesehatan dalam menangani berbagai isu terkait kepesertaan BPJS Kesehatan," terangnya.

# Lanjutnya, selama ini sistem data base kepesertaan Program JKN juga belum mampu merespon dinamika perubahan kependudukan secara real time. Selain permasalahan data kepesertaan, hingga saat ini pemerintah belum mencapai target Universal Health Coverage (UHC) sebagai bagian dari Sustainable Development Goals (SDGs) 3.8, yaitu “Mencapai UHC, termasuk proteksi risiko keuangan, akses pada layanan kesehatan dasar yang berkualitas, serta akses pada obat dan vaksin yang aman, efektif, berkualitas dan terjangkau bagi semua orang”.

# "Hal ini disebabkan karena BPJS Kesehatan belum melakukan upaya yang optimal dalam melakukan koordinasi dengan instansi terkait, antara lain untuk memastikan agar pemberi kerja melaporkan dan mendaftarkan tenaga kerjanya menjadi peserta Program JKN," ucap anggota DPD RI asal Sulawesi Selatan ini.

# Menanggapi itu, Direktur Kepesertaan BPJS Kesehatan David Bangun menjelaskan bahwa sampai dengan 1 Januari 2023 peserta JKN-KIS mencapai 249.660.154 jiwa. Data kepesertaan yang dikelola menggunakan NIK sebagai patokan data dan untuk memastikan tidak ada data ganda. Menurutnya data yang diterima adalah dari data kependudukan dan catatan sipil serta mekanismenya disinkronisasi secara web service dan web portal.

# "Ke depan melalui sinkronisasi data tidak ada delay informasi dan delay update data, sehingga iuran secara data real lebih valid, kami juga mengembangkan kanal layanan administrasi kepesertaan, informasi pengaduan baik secara tatap muka melalui kantor cabang, mobile CS, Mal Pelayanan Publik dan BPJS Satu. Selain itu pelayanan kami juga bisa diakses di Mobile JKN, BPJS Care Center, Pandawa dan website BPJS Kesehatan," jelas David Bangun.

# Terkait Iuran Program JKN, David Bangun memaparkan bahwa BPJS telah melakukan sesuai Peraturan Presiden No 64 Tahun 2020 dan Instruksi Presiden No. 1 Tahun 2022 Tentang Optimalisasi Pelaksanaan Program Jaminan Kesehatan Nasional.
# """

#### 10

In [ ]:
# # 10

# text_mentah = """
# KONTAN.CO.ID - JAKARTA. PT Golden Energy Mines Tbk (GEMS) mengincar volume penjualan 40-41 juta ton batubara di tahun 2023. Sekretaris Perusahaan GEMS, Sudin Sudiman mengatakan, sebanyak 65%-67% dari target penjualan tersebut dialokasikan untuk pasar ekspor.

# “Pangsa pasar Gems masih sama dengan tahun lalu, yang utama China, India Dan ASEAN,” tutur Sudin kepada Kontan.co.id, Rabu (1/2).

# Secara konsolidasi, GEMS sudah mengantongi restu dari Kementerian Energi dan Sumber Daya Mineral (ESDM) untuk memproduksi 40,3 juta ton batubara berdasarkan Rencana Kerja dan Anggaran Belanja (RKAB). Secara terperinci, rencana produksi konsolidasi tersebut berasal dari rencana produksi Borneo Indobara (BIB) 36 juta ton, Bara Sentosa Lestari (BSL) 2 juta ton, dan PT Kuansing Inti Makmut (KIM) 2,3 juta ton.

# Sebanyak 0,9 juta ton dari target penjualan 40-41 juta ton lainnya direncanakan direalisasi lewat skema perdagangan/trading. GEMS optimistis, prospek pasar ekspor batubara di tahun 2023 masih baik.

# “Prospek pasar ekspor masih bagus di 2023 meskipun harga sedikit menurun di kuartal I,” tutur Sudin.

# Belum ketahuan seperti apa realisasi produksi dan penjualan GEMS di tahun 2022. Sepanjang Januari-September 2022 lalu, GEMS telah membukukan penjualan 28,3 juta ton batubara. Jumlah tersebut naik 24% dibanding realisasi volume penjualan Januari-September 2021 yang berjumlah 22,9 juta ton.

# Seturut kenaikan volume, GEMS membukukan pendapatan US$ 2,06 miliar di Januari-September 2022, melesat 95,18% dibanding pendapatan GEMS di periode Januari-September 2021 yang sebesar US$ 1,05 miliar.

# Dari hasil pendapatan itu, GEMS mengantongi laba periode berjalan yang dapat diatribusikan kepada pemilik entitas induk alias laba bersih sebesar US$ 472,87 juta di Januari-September 2022. Jumlah tersebut naik 140,62% dibanding realisasi laba bersih Januari-September 2021 yang sebesar US$ 196,52 juta.
# """

## Predict Class

In [ ]:
new_text = re.sub(r".*- ", "", text_mentah)
# print(new_text)

sentences = []
start = 0
for match in re.finditer(r'\n', new_text):
    end = match.start()
    sentences.append(new_text[start:end])
    start = match.end()

# add the last sentence
sentences.append(new_text[start:])
sentences.remove(sentences[-1])
print(sentences)
print(len(sentences))

['Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.', '', 'Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.', '', '"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).', '', 'Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.', '', '"Kenapa di acara PDI Perjuangan bicara seperti i

In [ ]:
sentences

['Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.',
 '',
 'Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.',
 '',
 '"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).',
 '',
 'Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.',
 '',
 '"Kenapa di acara PDI Perjuangan bicara s

In [ ]:
# text = text_cleaning(sentences[0], True)
# text = word_tokenize(text)
# subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

# subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
# subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
# logits = model(subwords, subword_to_word_indices)[0]

# preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
# labels = [i2w[preds[i]] for i in range(len(preds))]

# hasil1 = pd.DataFrame({'words': text, 'label': labels})
# # hasil1

In [ ]:
# per paragraf predict

hasil = {"words":[],
         "label":[]
         }

for i in sentences:

  text = text_cleaning(i, True)
  text = word_tokenize(text)
  subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

  subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
  subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)

  try:
    logits = model(subwords, subword_to_word_indices)[0]

    preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
    labels = [i2w[preds[i]] for i in range(len(preds))]

    hasil["words"].append(text)
    hasil['label'].append(labels)
    # hasil = pd.DataFrame({'words': text, 'label': labels})

    hasil_full = pd.DataFrame(hasil)

  except:
    pass

word = [item for sublist in hasil_full['words'].tolist() for item in sublist]
label = [item for sublist in hasil_full['label'].tolist() for item in sublist]

hasil = pd.DataFrame({'words': word, 'label': label})

In [ ]:
hasil_full

,words,label
0,"[ketua, dpp, pdi, perjuangan, puan, maharani, ...","[O, O, B-ORGANISATION, O, B-PERSON, I-PERSON, ..."
1,"[puan, megawati, bicara, potensi, jokowi, mega...","[B-PERSON, I-PERSON, O, O, B-PERSON, O, O, O, ..."
2,"[wadah, maksudnya, ayo, maju, ayo, bakti, laku...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[puan, ketua, pdi, p, megawati, menempatkan, k...","[B-PERSON, O, B-ORGANISATION, O, B-PERSON, O, ..."
4,"[acara, pdi, perjuangan, bicara, acara, intern...","[O, B-ORGANISATION, I-ORGANISATION, O, O, O, O..."
5,"[puan, megawati, merendahkan, jokowi, nasib, j...","[B-PERSON, I-PERSON, O, O, O, O, O, O, O]"
6,"[ketua, dpr, ri, bilang, ibundanya, menyayangi...","[O, B-ORGANISATION, I-ORGANISATION, O, O, O, O..."
7,"[mega, sayang, banget, jokowi, hormat, jokowi,...","[O, O, O, O, O, O, O, O]"
8,"[puan, lantas, menyinggung, pidato, megawati, ...","[B-PERSON, I-PERSON, O, O, B-PERSON, O, O, B-P..."
9,"[mega, mengaku, pasang, badan, dirundung, joko...","[B-PERSON, O, O, O, O, B-PERSON, O, O]"


In [ ]:
hasil

,words,label
0,ketua,O
1,dpp,O
2,pdi,B-ORGANISATION
3,perjuangan,O
4,puan,B-PERSON
...,...,...
266,beliau,O
267,presiden,O
268,legal,O
269,formal,O


In [ ]:
# hasil.to_csv('/content/drive/MyDrive/nolimit/NER/model/hasil_tes.csv', index=False)

In [ ]:
# ini yang satuan

# text = text_cleaning(text_mentah, True)
# text = word_tokenize(text)
# subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

# subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
# subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
# logits = model(subwords, subword_to_word_indices)[0]

# preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
# labels = [i2w[preds[i]] for i in range(len(preds))]

# hasil = pd.DataFrame({'words': text, 'label': labels})

In [ ]:
# kalau udha dapat hasil di df nya maka kesini

hasil = hasil.drop_duplicates().reset_index(drop=True)
hasil = hasil[hasil['label'] != 'O'].reset_index(drop=True)

In [ ]:
hasil

,words,label
0,pdi,B-ORGANISATION
1,puan,B-PERSON
2,maharani,I-PERSON
3,megawati,B-PERSON
4,soekarnoputri,I-PERSON
5,joko,B-PERSON
6,widodo,I-PERSON
7,partai,I-ORGANISATION
8,banteng,I-ORGANISATION
9,megawati,I-PERSON


In [ ]:
#full dibawah ini

# print(text_mentah)
print('-'*100)

output = {}

for i in range(len(hasil)):

  # --------------- PERSON -----------------------------
  try:
    if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]!='I-PERSON':
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'
    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PERSON' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'

    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON' and hasil['label'][i+2]=='I-PERSON':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PERSON')
      output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PERSON'
    elif hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON':
      print(hasil['words'][i],hasil['words'][i+1], '=> PERSON')
      output[hasil['words'][i],hasil['words'][i+1]] = 'PERSON'
    else:
      pass

  except:
    pass


  # --------------- PLACE -----------------------------
  try:
    if hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]!='I-PLACE':
      print(hasil['words'][i], '=> PLACE')
      output[hasil['words'][i]] = 'PLACE'
    else:
      pass

  except:
    if hasil['label'][i]=='B-PLACE' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PLACE')
      output[hasil['words'][i]] = 'PLACE'
    else:
      pass

  try:
    if hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]=='I-PLACE' and hasil['label'][i+2]=='I-PLACE':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PLACE')
      output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PLACE'
    elif hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]=='I-PLACE':
      print(hasil['words'][i],hasil['words'][i+1], '=> PLACE')
      output[hasil['words'][i],hasil['words'][i+1]] = 'PLACE'
    else:
      pass

  except:
    pass

  # --------------- ORGANISATION ----------------------------
  try:
    if hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]!='I-ORGANISATION':
      print(hasil['words'][i], '=> ORGANISATION')
      output[hasil['words'][i]] = 'ORGANISATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-ORGANISATION' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> ORGANISATION')
      output[hasil['words'][i]] = 'ORGANISATION'
    else:
      pass

  try:
    if hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]=='I-ORGANISATION' and hasil['label'][i+2]=='I-ORGANISATION':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> ORGANISATION')
      output[hasil['words'][i],hasil['words'][i+1]],hasil['words'][i+2] = 'ORGANISATION'
    elif hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]=='I-ORGANISATION':
      print(hasil['words'][i],hasil['words'][i+1], '=> ORGANISATION')
      output[hasil['words'][i],hasil['words'][i+1]] = 'ORGANISATION'
    else:
      pass

  except:
    pass

  # --------------- O -----------------------------
  try:
    if hasil['label'][i]=='O':
      print(hasil['words'][i], '=> O')
      output[hasil['words'][i]] = 'O'
    else:
      pass

  except:
    pass

# for x,y in enumerate(text):
#   if bool(re.findall(r"(?=("+'|'.join(kata)+r"))", y)) == True:
#     #print(y)

#     if y == 'kata':
#       print(' '.join([str(elem) for elem in text]).split('kata')[0].strip())

----------------------------------------------------------------------------------------------------
pdi => ORGANISATION
puan maharani => PERSON
megawati soekarnoputri => PERSON
joko widodo => PERSON
jokowi => PERSON
rosi => PERSON
partai => ORGANISATION
mega => PERSON
dpr ri => ORGANISATION
bu mega => PERSON
dki jakarta => PLACE
jakarta => PLACE
kemayoran => PLACE
seloroh => PERSON


In [ ]:
# # ini coba yang baru

# output = {}

# for i in range(len(hasil)):

#   # --------------- PERSON -----------------------------
#   try:
#     if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]!='I-PERSON':
#       print(hasil['words'][i], '=> PERSON')
#       output[hasil['words'][i]] = 'PERSON'
#     else:
#       pass

#   except:
#     pass

#   try:
#     if hasil['label'][i]=='B-PERSON' and ((i+1)==len(hasil)):
#       print(hasil['words'][i], '=> PERSON')
#       output[hasil['words'][i]] = 'PERSON'

#     else:
#       pass

#   except:
#     pass

#   try:
#     if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON' and hasil['label'][i+2]=='I-PERSON':
#       print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PERSON')
#       output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PERSON'
#     elif hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON':
#       print(hasil['words'][i],hasil['words'][i+1], '=> PERSON')
#       output[hasil['words'][i],hasil['words'][i+1]] = 'PERSON'
#     else:
#       pass

#   except:
#     pass

In [ ]:
# output

In [ ]:
output.items()

dict_items([('pdi', 'ORGANISATION'), (('puan', 'maharani'), 'PERSON'), (('megawati', 'soekarnoputri'), 'PERSON'), (('joko', 'widodo'), 'PERSON'), ('jokowi', 'PERSON'), ('rosi', 'PERSON'), ('partai', 'ORGANISATION'), ('mega', 'PERSON'), (('dpr', 'ri'), 'ORGANISATION'), (('bu', 'mega'), 'PERSON'), (('dki', 'jakarta'), 'PLACE'), ('jakarta', 'PLACE'), ('kemayoran', 'PLACE'), ('seloroh', 'PERSON')])

In [ ]:
new_output = {}
for k,v in output.items():
  if type(k) == tuple:
    new_output[' '.join(k)] = v

  else:
    new_output[''.join(k)] = v

# new_output

df = pd.DataFrame()

df['text'] = new_output.keys()
df['label'] = new_output.values()

In [ ]:
df['label'].value_counts()

PERSON          8
ORGANISATION    3
PLACE           3
Name: label, dtype: int64

### Hasil

In [ ]:
print(df[df['label'] == 'PERSON'])
print(df[df['label'] == 'PLACE'])
print(df[df['label'] == 'ORGANISATION'])

                      text   label
1            puan maharani  PERSON
2   megawati soekarnoputri  PERSON
3              joko widodo  PERSON
4                   jokowi  PERSON
5                     rosi  PERSON
7                     mega  PERSON
9                  bu mega  PERSON
13                 seloroh  PERSON
           text  label
10  dki jakarta  PLACE
11      jakarta  PLACE
12    kemayoran  PLACE
     text         label
0     pdi  ORGANISATION
6  partai  ORGANISATION
8  dpr ri  ORGANISATION


In [ ]:
# df.to_csv('/content/drive/MyDrive/nolimit/NER/model/hasil_model3do.csv', index=False)

## Predict Proba

In [ ]:
new_text = re.sub(r".*- ", "", text_mentah)
# print(new_text)

sentences = []
start = 0
for match in re.finditer(r'\n', new_text):
    end = match.start()
    sentences.append(new_text[start:end])
    start = match.end()

# add the last sentence
sentences.append(new_text[start:])
sentences.remove(sentences[-1])
print(sentences)
print(len(sentences))

['Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.', '', 'Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.', '', '"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).', '', 'Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.', '', '"Kenapa di acara PDI Perjuangan bicara seperti i

In [ ]:
# per paragraf predict

hasil = {"words":[],
         "label":[]
         }

for i in sentences:

  text = text_cleaning(i, True)
  text = word_tokenize(text)
  subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

  subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
  subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)

  try:
    logits = model(subwords, subword_to_word_indices)[0]

    preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
    labels = [i2w[preds[i]] for i in range(len(preds))]

    hasil["words"].append(text)
    hasil['label'].append(labels)
    # hasil = pd.DataFrame({'words': text, 'label': labels})

    hasil_full = pd.DataFrame(hasil)

  except:
    pass

word = [item for sublist in hasil_full['words'].tolist() for item in sublist]
label = [item for sublist in hasil_full['label'].tolist() for item in sublist]

hasil = pd.DataFrame({'words': word, 'label': label})

In [ ]:
prob

tensor([[[6.4746e-21, 4.2191e-17, 1.4871e-18, 2.0433e-10, 1.3137e-10,
          8.2974e-02, 3.0406e-21],
         [5.9463e-21, 5.6480e-17, 1.4152e-18, 2.1060e-10, 1.1887e-10,
          8.1811e-02, 3.1043e-21],
         [4.2698e-13, 9.9994e-01, 3.7998e-10, 3.3535e-04, 9.4790e-05,
          4.2999e-16, 4.4829e-14],
         [9.9113e-17, 2.6390e-14, 1.3912e-07, 6.6072e-08, 1.1556e-05,
          9.4451e-10, 4.4539e-18],
         [7.0019e-10, 6.6614e-06, 7.1133e-08, 1.0770e-01, 6.0667e-02,
          7.8975e-21, 3.7755e-01],
         [4.0125e-01, 6.7647e-06, 5.4587e-08, 2.7470e-01, 7.6255e-02,
          5.9517e-21, 2.7241e-10],
         [7.2259e-21, 3.7828e-17, 1.5910e-18, 1.9818e-10, 1.3826e-10,
          8.3589e-02, 2.9465e-21],
         [6.3957e-21, 3.9516e-17, 1.4820e-18, 2.0983e-10, 1.3921e-10,
          8.3947e-02, 3.0658e-21],
         [5.9923e-10, 6.3748e-06, 6.8715e-08, 1.2840e-01, 5.4874e-02,
          8.4486e-21, 3.7236e-01],
         [3.3360e-01, 5.5213e-06, 4.5510e-08, 2.3978e-0

In [ ]:
# coba cari proba nya

hasil = {"words":[],
         "label":[],
         "proba":[]
         }

text = 'Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.'
text = text_cleaning(text, True)
text = word_tokenize(text)
subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)

import torch.nn.functional as nnf

try:
  logits = model(subwords, subword_to_word_indices)[0]

  prob = nnf.softmax(logits, dim=1)

  top_p, top_class = prob.topk(k=1, dim = -1)
  top_class = top_class.reshape(-1).numpy()

  # labels = [i2w[top_class[i]] for i in range(len(top_class))]
  proba = top_p.reshape(-1).tolist()

  preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
  labels = [i2w[preds[i]] for i in range(len(preds))]

  hasil["words"].append(text)
  hasil['label'].append(labels)
  hasil['proba'].append(proba)

  # hasil = pd.DataFrame({'words': text, 'label': labels, 'proba': top_p})

  hasil_full = pd.DataFrame(hasil)

except:
  pass

In [ ]:
hasil_full

,words,label,proba
0,"[ketua, dpp, pdi, perjuangan, puan, maharani, ...","[O, O, B-ORGANISATION, O, B-PERSON, I-PERSON, ...","[0.08297403901815414, 0.08181145042181015, 0.9..."


In [ ]:
word = [item for sublist in hasil_full['words'].tolist() for item in sublist]
label = [item for sublist in hasil_full['label'].tolist() for item in sublist]
proba = [item for sublist in hasil_full['proba'].tolist() for item in sublist]

hasil = pd.DataFrame({'words': word, 'label': label, 'proba': proba})

In [ ]:
hasil

,words,label,proba
0,ketua,O,0.082974
1,dpp,O,0.081811
2,pdi,B-ORGANISATION,0.999935
3,perjuangan,O,0.000012
4,puan,B-PERSON,0.377553
5,maharani,I-PERSON,0.401251
6,maksud,O,0.083589
7,pidato,O,0.083947
8,megawati,B-PERSON,0.372362
9,soekarnoputri,I-PERSON,0.333600


In [ ]:
# import torch.nn.functional as nnf

# prob = nnf.softmax(logits, dim=1)

# top_p, top_class = prob.topk(k=1, dim = -1)
# top_p = top_p.reshape(-1).tolist()
# top_class = top_class.reshape(-1).numpy()

In [ ]:
# convert_class = [i2w[top_class[i]] for i in range(len(top_class))]

# for p,c in zip(top_p[0].tolist(), top_class[0].tolist()):
#   print(c)
#   for num in p:
#     print("{:f}".format(num))

In [ ]:
# kalau udha dapat hasil di df nya maka kesini

hasil = hasil.drop_duplicates().reset_index(drop=True)
hasil = hasil[hasil['label'] != 'O'].reset_index(drop=True)

In [ ]:
hasil

,words,label,proba
0,pdi,B-ORGANISATION,0.999935
1,puan,B-PERSON,0.377553
2,maharani,I-PERSON,0.401251
3,megawati,B-PERSON,0.372362
4,soekarnoputri,I-PERSON,0.333600
5,joko,B-PERSON,0.250085
6,widodo,I-PERSON,0.265140
7,partai,I-ORGANISATION,0.043501
8,banteng,I-ORGANISATION,0.991344


In [ ]:
#full dibawah ini

# print(text_mentah)
print('-'*100)

output = {}

for i in range(len(hasil)):

  # --------------- PERSON -----------------------------
  try:
    if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]!='I-PERSON':
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'
    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PERSON' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PERSON')
      output[hasil['words'][i]] = 'PERSON'

    else:
      pass

  except:
    pass

  try:
    if hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON' and hasil['label'][i+2]=='I-PERSON':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PERSON')
      output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PERSON'
    elif hasil['label'][i]=='B-PERSON' and hasil['label'][i+1]=='I-PERSON':
      print(hasil['words'][i],hasil['words'][i+1], '=> PERSON')
      output[hasil['words'][i],hasil['words'][i+1]] = 'PERSON'
    else:
      pass

  except:
    pass


  # --------------- PLACE -----------------------------
  try:
    if hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]!='I-PLACE':
      print(hasil['words'][i], '=> PLACE')
      output[hasil['words'][i]] = 'PLACE'
    else:
      pass

  except:
    if hasil['label'][i]=='B-PLACE' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> PLACE')
      output[hasil['words'][i]] = 'PLACE'
    else:
      pass

  try:
    if hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]=='I-PLACE' and hasil['label'][i+2]=='I-PLACE':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> PLACE')
      output[hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2]] = 'PLACE'
    elif hasil['label'][i]=='B-PLACE' and hasil['label'][i+1]=='I-PLACE':
      print(hasil['words'][i],hasil['words'][i+1], '=> PLACE')
      output[hasil['words'][i],hasil['words'][i+1]] = 'PLACE'
    else:
      pass

  except:
    pass

  # --------------- ORGANISATION ----------------------------
  try:
    if hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]!='I-ORGANISATION':
      print(hasil['words'][i], '=> ORGANISATION')
      output[hasil['words'][i]] = 'ORGANISATION'
    else:
      pass

  except:
    if hasil['label'][i]=='B-ORGANISATION' and ((i+1)==len(hasil)):
      print(hasil['words'][i], '=> ORGANISATION')
      output[hasil['words'][i]] = 'ORGANISATION'
    else:
      pass

  try:
    if hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]=='I-ORGANISATION' and hasil['label'][i+2]=='I-ORGANISATION':
      print(hasil['words'][i],hasil['words'][i+1],hasil['words'][i+2], '=> ORGANISATION')
      output[hasil['words'][i],hasil['words'][i+1]],hasil['words'][i+2] = 'ORGANISATION'
    elif hasil['label'][i]=='B-ORGANISATION' and hasil['label'][i+1]=='I-ORGANISATION':
      print(hasil['words'][i],hasil['words'][i+1], '=> ORGANISATION')
      output[hasil['words'][i],hasil['words'][i+1]] = 'ORGANISATION'
    else:
      pass

  except:
    pass

  # --------------- O -----------------------------
  try:
    if hasil['label'][i]=='O':
      print(hasil['words'][i], '=> O')
      output[hasil['words'][i]] = 'O'
    else:
      pass

  except:
    pass

# for x,y in enumerate(text):
#   if bool(re.findall(r"(?=("+'|'.join(kata)+r"))", y)) == True:
#     #print(y)

#     if y == 'kata':
#       print(' '.join([str(elem) for elem in text]).split('kata')[0].strip())

----------------------------------------------------------------------------------------------------
pdi => ORGANISATION
puan maharani => PERSON
megawati soekarnoputri => PERSON
joko widodo => PERSON


In [ ]:
# output

In [ ]:
output.items()

dict_items([('pdi', 'ORGANISATION'), (('puan', 'maharani'), 'PERSON'), (('megawati', 'soekarnoputri'), 'PERSON'), (('joko', 'widodo'), 'PERSON')])

In [ ]:
new_output = {}
for k,v in output.items():
  if type(k) == tuple:
    new_output[' '.join(k)] = v

  else:
    new_output[''.join(k)] = v

# new_output

df = pd.DataFrame()

df['text'] = new_output.keys()
df['label'] = new_output.values()

In [ ]:
df['label'].value_counts()

PERSON          3
ORGANISATION    1
Name: label, dtype: int64

### Hasil

In [ ]:
print(df[df['label'] == 'PERSON'])
print(df[df['label'] == 'PLACE'])
print(df[df['label'] == 'ORGANISATION'])

                     text   label
1           puan maharani  PERSON
2  megawati soekarnoputri  PERSON
3             joko widodo  PERSON
Empty DataFrame
Columns: [text, label]
Index: []
  text         label
0  pdi  ORGANISATION


In [ ]:
# df.to_csv('/content/drive/MyDrive/nolimit/NER/model/hasil_model3do.csv', index=False)

# Statement

## **untuk statement**

In [ ]:
kata = [
        'Ujar',
        'Kata',
        'Ucap',
        'Mengatakan',
        'Mengucapkan',
        'Menugaskan',
        'Bertanya',
        'Berkata',
        'Jawab',
        'Perintah',
        'Memerintahkan',
        'Menugaskan',
        'Menanyakan',
        'Berkata',
        'Berucap',
        'Menyuruh',
        'Memanggil',
        'Cetus',
        'Celetuk',
        'Menyambut',
        'Sambut',
        'Menyanggah',
        'Sanggah',
        'Menyapa',
        'Sapa',
        'Tegur',
        'Menegur',
        'Harap',
        'Balas',
        'Membalas',
        'Potong',
        'Memotong',
        'Usul',
        'Mengusulkan',
        'Menyampaikan',
        'Tolak',
        'Menolaknya',
        'Ungkap',
        'Mengungkapkan',
        'Sahut',
        'Lanjut',
        'Saran',
        'Menyarankan',
        'Menyela'
        ]

In [ ]:
kata = [x.lower() for x in kata]

In [ ]:
print(kata)

**bismillah dibawah bisa untuk statement**

- list of paragraf
- list of statement/kalimat
- detect statement
- dapat people dan statement
- statement nya sentimen kan

**tahap flow NER & Statement:** *Q1*

- artikel
- paragraf
- kalimat
- detect statement
- extract statement + entity
- classification sentiment

In [ ]:
# artikel puan contoh

text_mentah = """JAKARTA, KOMPAS.com - Ketua DPP PDI Perjuangan Puan Maharani menjelaskan maksud dari pidato Megawati Soekarnoputri di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang menyinggung nasib Presiden Joko Widodo jika tak ada partai banteng.

Menurut Puan, Megawati bicara demikian karena dia melihat potensi besar di diri Jokowi. Mega tengah mengibaratkan PDI-P sebagai wadah bagi para kader, termasuk Jokowi, untuk berjuang.

"Jadi ini sebagai wadah. Jadi maksudnya, ayo dong kita sama-sama maju, ayo dong sama-sama kita memberikan bakti kita, atau apa yang bisa kita lakukan untuk bisa membangun bangsa dan negara. Kan diperlukan, it takes two to tango untuk bisa melaksanakan hal itu," kata Puan dalam acara Rosi Kompas TV, dikutip Jumat (13/1/2023).

Puan mengatakan, sebagai ketua umum PDI-P, Megawati selalu menempatkan diri sebagai ibu bagi kader-kader partai banteng. Dia pun menilai, pernyataan Mega itu wajar diucapkan seorang ketua umum partai.

"Kenapa di acara PDI Perjuangan bicara seperti itu, karena ini acara internal, dianggap semua (kader) itu anak-anaknya," ujarnya.

Puan mengatakan, Megawati tak bermaksud merendahkan Jokowi dengan mengatakan nasib Jokowi mungkin berbeda jika PDI-P tak ada.

Sebaliknya, Ketua DPR RI itu bilang, ibundanya sangat menyayangi dan menghormati Jokowi.

"Kalau tahu, Ibu Mega itu sayang banget sama Pak Jokowi. Dan tentu saja hormat sekali sama Pak Jokowi sebagai presiden," kata Puan.

Puan lantas menyinggung pidato Megawati beberapa waktu lalu yang menyebut bahwa dia menangis karena Jokowi terus menerus dirundung.

Mega bahkan mengaku siap pasang badan, tak apa dia dirundung asalkan bukan bukan Jokowi yang kena bully.

Menurut Puan, sikap Megawati itu menunjukkan betapa dia sangat menyayangi dan menghormati Jokowi sebagai adiknya. Bahwa Megawati ingin Jokowi sukses menjalankan tugas-tugasnya sebagai kepala negara.

"Saya baru pertama kali ini mendengar Bu Mega membela seseorang sampai seperti itu," ucap Puan.

"Bahkan (Megawati) sampai (mengatakan), nggak papa saya di- bully karena saya mau membela Pak Jokowi. Itu kalau bukan sayang dan hormat apa," tuturnya.

Lagi pula, lanjut Puan, dalam pidatonya Megawati juga sempat memuji Jokowi dengan menyebut bahwa mantan Gubernur DKI Jakarta itu pintar sehingga PDI-P bersedia mengusungnya jadi pemimpin.

Oleh karenanya, Puan menegaskan bahwa Megawati tak bermaksud buruk, apalagi menghina Jokowi lewat pidatonya.

"Jadi bukan ada yang rasa yang mau merendahkan atau sepertinya nggak hormat, itu justru kebalikan," tutur mantan Menteri Koordinator Bidang Pembangunan Manusia dan Kebudayaan (Menko PMK) itu.

Sebelumnya, nama Presiden Jokowi berulang kali disinggung oleh Megawati dalam pidatonya di acara Hari Ulang Tahun (HUT) ke-50 PDI-P yang digelar di Jakarta International Expo, Kemayoran, Jakarta Pusat, Selasa (10/1/2023).

Di hadapan ribuan kader yang hadir, Megawati berkelakar, nasib Jokowi tidak akan sama seperti sekarang jika tak ada PDI-P.

"Pak Jokowi itu kayak begitu lho, mentang-mentang. Lah iya, padahal Pak Jokowi kalau enggak ada PDI Perjuangan juga, aduh, kasihan dah," kata Megawati sambil tertawa.

Seloroh Megawati tersebut disambut tawa para tamu undangan, termasuk Jokowi yang duduk di barisan kursi tamu terdepan.

Presiden kelima RI itu bilang, seandainya PDI-P tak memberikan dukungan, maka Jokowi tidak akan jadi presiden seperti sekarang.

"Lho legal formal lho, beliau jadi presiden itu enggak ada kan ini, legal formal diikuti terus sama saya," ujarnya.

Pernyataan Megawati itu pun memunculkan kritik dari sejumlah kalangan. Putri Soekarno itu bahkan dianggap merendahkan presiden."""

## input artikel

In [ ]:
import re

In [ ]:
text_mentah = """Garut: Gempa berkekuatan magnitudo 4,3 pada Rabu, 1 Februari pukul 22.57 WIB telah menyebabkan 495 rumah mengalami kerusakan tepatnya berada di Kecamatan Pasirwangi dan Kecamatan Samarang, Kabupaten Garut. Kejadian tersebut tidak menyebabkan korban jiwa tetapi, petugas gabungan TNI, Polri dan BPBD masih terus melakukan pendataan.

Berdasarkan data laporan BPBD Kabupaten Garut, hingga Kamis malam tidak ada korban jiwa. Tercatat total rumah terdampak sebanyak 495 unit rumah, terdiri dari 367 unit rumah mengalami rusak ringan (RR), 111 unit rumah rusak sedang (RS) dan 17 unit rumah rusak berat, serta 8 unit fasilitas pendidikan mengalami kerusakan.

Kepala Pelaksana Badan Penanggulangan Bencana Daerah Kabupaten Garut, Satria Budi mengatakan, gempa bumi berkekuatan 4,3 magnitudo terletak pada koordinat 7.27 LS dan 107.73 BT, atau tepatnya berlokasi di darat pada jarak 19 kilometer Barat Daya Kabupaten Garut pada kedalaman 3 kilometer.

"Berdasarkan rilis BMKG, dampak gempa bumi yang digambarkan peta tingkat guncangan (Shakemap) BMKG dan berdasarkan laporan dari masyarakat dirasakan di wilayah GarutÂ Pasirwangi, Cisurupan, Bayongbong, Samarang dengan Skala Intensitas IV MMI. Dengan memperhatikan lokasi episenter dan kedalaman hiposenternya, gempabumi yang terjadi merupakan jenis gempabumi dangkal akibat aktivitas Sesar Garsela," katanya, Jumat, 3 Februari 2023.

Sementara Wakil Bupati Garut, Helmi Budiman meminta masyarakat untuk tetap tenang. Petugas gabungan TNI, Polri dan BPBD sedang melakukan verifikasi di lapangan.

"Secara umum untuk di Kecamatan Samarang lebih sedikit tapi kondisinya berat dan sedang, tapi di Pasirwangi jumlahnya memang banyak 370 tapi ringan dan sedang. Jadi, sekarang ini sedang melakukan verifikasi dulu dan personil yang melakukan assessment di lapangan, kita panggil di BPBD dan nanti akan melakukan pembahasan sehingga nanti tindaklanjut akan dilakukan bisa semakin jelas," ujarnya.

Ia berharap masyarakat yang rumahnya mengalami rusak berat untuk sementara waktu supaya berpindah ke lokasi lebih aman, karena ditakutkan gempa susulan yang terjadi.

"Jadi, harap tenang dan siap-siap ya kalau ada gempa susulan sekali lagi rumah yang rusak berat lah ya untuk tidak ditinggali dulu, dan tentu yang bisa kita lakukan yang terbaik itu lakukanlah, bisa di keluarga, bisa di tetangga atau di tempat yang lain yang aman," paparnya.
"""

**1. Extract paragraf dari artikel**



In [ ]:
new_text = re.sub(r".*- ", "", text_mentah)
# print(new_text)

sentences = []
start = 0
for match in re.finditer(r'\n', new_text):
    end = match.start()
    sentences.append(new_text[start:end])
    start = match.end()

# add the last sentence
sentences.append(new_text[start:])
sentences.remove(sentences[-1])
print(sentences)
print(len(sentences))

['Garut: Gempa berkekuatan magnitudo 4,3 pada Rabu, 1 Februari pukul 22.57 WIB telah menyebabkan 495 rumah mengalami kerusakan tepatnya berada di Kecamatan Pasirwangi dan Kecamatan Samarang, Kabupaten Garut. Kejadian tersebut tidak menyebabkan korban jiwa tetapi, petugas gabungan TNI, Polri dan BPBD masih terus melakukan pendataan.', 'Berdasarkan data laporan BPBD Kabupaten Garut, hingga Kamis malam tidak ada korban jiwa. Tercatat total rumah terdampak sebanyak 495 unit rumah, terdiri dari 367 unit rumah mengalami rusak ringan (RR), 111 unit rumah rusak sedang (RS) dan 17 unit rumah rusak berat, serta 8 unit fasilitas pendidikan mengalami kerusakan.', '', 'Kepala Pelaksana Badan Penanggulangan Bencana Daerah Kabupaten Garut, Satria Budi mengatakan, gempa bumi berkekuatan 4,3 magnitudo terletak pada koordinat 7.27 LS dan 107.73 BT, atau tepatnya berlokasi di darat pada jarak 19 kilometer Barat Daya Kabupaten Garut pada kedalaman 3 kilometer.', '', '"Berdasarkan rilis BMKG, dampak gempa 

In [ ]:
for i, y in enumerate(sentences):
  if len(y) < 10:
    sentences.remove(sentences[i])
  else:
    pass

for i, y in enumerate(sentences):
  print(i, '-', y)

0 - Garut: Gempa berkekuatan magnitudo 4,3 pada Rabu, 1 Februari pukul 22.57 WIB telah menyebabkan 495 rumah mengalami kerusakan tepatnya berada di Kecamatan Pasirwangi dan Kecamatan Samarang, Kabupaten Garut. Kejadian tersebut tidak menyebabkan korban jiwa tetapi, petugas gabungan TNI, Polri dan BPBD masih terus melakukan pendataan.
1 - Berdasarkan data laporan BPBD Kabupaten Garut, hingga Kamis malam tidak ada korban jiwa. Tercatat total rumah terdampak sebanyak 495 unit rumah, terdiri dari 367 unit rumah mengalami rusak ringan (RR), 111 unit rumah rusak sedang (RS) dan 17 unit rumah rusak berat, serta 8 unit fasilitas pendidikan mengalami kerusakan.
2 - Kepala Pelaksana Badan Penanggulangan Bencana Daerah Kabupaten Garut, Satria Budi mengatakan, gempa bumi berkekuatan 4,3 magnitudo terletak pada koordinat 7.27 LS dan 107.73 BT, atau tepatnya berlokasi di darat pada jarak 19 kilometer Barat Daya Kabupaten Garut pada kedalaman 3 kilometer.
3 - "Berdasarkan rilis BMKG, dampak gempa bum

In [ ]:
# sentence = sentences[7].lower()
# sentence

In [ ]:
#mengatakan
people = 'aku'
match = re.search(r"(.*?) mengatakan(.*)", sentence)
if match:
  if 'itu' in match.group(1):
    people = people

  elif 'yang' in match.group(1):
    people = ''
    statement = ''

  elif 'juga' in match.group(1):
    people = match.group(1).split()
    people.remove('juga')
    people = ' '.join(people[-2:])

    statement = match.group(2)
    if len(statement) < 20:
      statement = ''
      people = people

  else:
    people = match.group(1)
    if len(people) > 20:
      comma_index = people.rfind(',')
      people = people[comma_index+1:].strip()

    if len(people) > 20:
      split = people.split()
      people = " ".join(split[:2])

    if len(people) > 15:
      people = match.group(1)
      people = people[comma_index+1:].strip()
      split = people.split()
      people = " ".join(split[-1:])

      if 'juga' in people:
        people = match.group(1)
        people = people[comma_index+1:].strip()
        split = people.split()
        people = " ".join(split[:2])

    statement = match.group(2)
    if len(statement) < 20:
      statement = ''
      people = people

  print(people, "==>", statement)

  people = people

In [ ]:
len('sri mulyani')

11

## **dibawah ini udah mulai bisa FIX**

In [ ]:
# coba generate statement pakai kalimat dari paragraf aja

# get yang mau di ambil
# sentence = sentences[21].lower()
# print(sentence)
# print('----------'*30)

sentences = [x.lower() for x in sentences]
people = ''

for sentence in sentences:
  #menurut
  match = re.search(r"menurut (.*?),", sentence)
  if match:
    statement = sentence.split(match.group(0))[1]

    if 'dia' not in match.group(1):
      people = match.group(1)
    elif 'dia' in match.group(1):
      people = ''
      statement = ''

    if len(people) > 20:
      people = ''
      statement = ''
    else:
      pass

    print(people, "==>", statement)

  #menurut
  match = re.search(r"\bmenurut\b", sentence)

  if match:
      statement = sentence[:match.start()].strip()

      if 'dia' not in sentence[match.end():].strip():
        people = sentence[match.end():].strip()
      elif 'dia' in sentence[match.end():].strip():
        people = people
        statement = ''

      if len(people) > 20:
        people = ''
        statement = ''
      else:
        pass

      if len(people) < 10:
        people = ''
        statement = ''

      # people = sentence[match.end():].strip()
      print(people, "==>", statement)

  #kata
  match = re.search(r"kata (.*)", sentence)
  if match:
    if 'dia' not in match.group(1).split(" ")[0]:
      people = match.group(1).split(" ")[0]
    elif 'dia' in match.group(1).split(" ")[0]:
      people = people

    if people.endswith("."):
      people = people[:-1]

    statement = sentence.split(match.group(0))[0]
    if len(statement) < 20:
      try:
        statement = sentence.split(",", 1)[1]

      except:
        pass

    if 'lain' not in match.group(1).split(" ")[0]:
      people = match.group(1).split(" ")[0]
    elif 'lain' in match.group(1).split(" ")[0]:
      people = ''
      statement = ''

    if 'dia' not in people:
      people = people
    elif 'dia' in people:
      people = ''
      statement = ''

    if len(statement) < 20:
      people = ''
      statement = ''
    else:
      pass

    print(people, "==>", statement)

  # #jelas
  # match = re.search(r"\bjelas\b", sentence)
  # if match:
  #   statement = sentence[start_index:end_index].strip()

  #   start_index = match.end()
  #   end_index = len(sentence)
  #   if len(sentence[start_index:end_index].strip())<20:
  #     people = ''
  #     statement = ''
  #   else:
  #     people = sentence[start_index:end_index].strip()

  #   start_index = 0
  #   end_index = match.start()

  #   print(people, "==>", statement)

  #jelas
  match = re.search(r"\bjelas\b", sentence)
  if match:
    start_index = 0
    end_index = match.start()

    statement = sentence[start_index:end_index].strip()

    start_index = match.end()
    end_index = len(sentence)
    if len(sentence[start_index:end_index].strip())>20:
      people = ''
      statement = ''
    else:
      people = sentence[start_index:end_index].strip()

    print(people, "==>", statement)

  #mengatakan
  match = re.search(r"(.*?) mengatakan(.*)", sentence)
  if match:
    if 'itu' in match.group(1):
      people = people

    elif 'yang' in match.group(1):
      people = ''
      statement = ''

    elif 'juga' in match.group(1):
      people = match.group(1).split()
      people.remove('juga')
      people = ' '.join(people[-2:])

      statement = match.group(2)
      if len(statement) < 20:
        statement = ''
        people = people

    else:
      people = match.group(1)
      if len(people) > 20:
        comma_index = people.rfind(',')
        people = people[comma_index+1:].strip()

      if len(people) > 20:
        split = people.split()
        people = " ".join(split[:2])

      if len(people) > 15:
        people = match.group(1)
        people = people[comma_index+1:].strip()
        split = people.split()
        people = " ".join(split[-1:])

        if 'juga' in people:
          people = match.group(1)
          people = people[comma_index+1:].strip()
          split = people.split()
          people = " ".join(split[:2])

      statement = match.group(2)
      if len(statement) < 20:
        statement = ''
        people = people

    print(people, "==>", statement)

    people = people

  #ujarnya
  match = re.search(r"(.*?) ujarnya\.", sentence)
  if match:
    # people = people.mode()
    people = people
    statement = match.group(1)
    print(people, "==>", statement)

  #sebaliknya
  match = re.search(r"sebaliknya,(.*)", sentence)
  if match:
    # people = people.mode()
    after_keyword = match.group(1)
    statement = after_keyword.split(",", 1)[1]
    print(people, "==>", statement)

  #ucap
  match = re.search(r"ucap (.*)", sentence)
  if match:
    if 'dia' not in match.group(1).split(" ")[0]:
      people = match.group(1).split(" ")[0]
    elif 'dia' in match.group(1).split(" ")[0]:
      people = people

    if people.endswith("."):
      people = people[:-1]
    statement = sentence.split(match.group(0))[0]
    print(people, "==>", statement)

  #tuturnya
  match = re.search(r"(.*?) tuturnya\.", sentence)
  if match:
    # people = people.mode()
    statement = match.group(1)
    print(people, "==>", statement)

  #lanjut
  match = re.search(r"lanjut (.*?),", sentence)
  if match:
      people = match.group(1)
      statement = sentence.split(people)[1]
      print(people, "==>", statement)

  #menegaskan
  match = re.search(r"menegaskan", sentence)
  if match:
    words = sentence.split("menegaskan")

    if len(words[0].split(", ")[-1].split()) >= 5:
      people = people
    else:
      people = words[0].split(", ")[-1]

    if 'ia' in words[0].split(", ")[-1]:
      people = people
    else:
      pass

    if people == '':
      people = ' '.join(words[0].split(", ")[-1].split()[-3:])

    # people = words[0].split(", ")[-1]
    statement = words[1]
    print(people, "==>", statement)

  #menuturkan
  match = re.search(r"menuturkan (.*)", sentence)
  if match:
    statement = match.group(1)
    people = sentence.split("menuturkan")[0]
    if "," in people:
      people = people.split(",", 1)[1]
    print(people, "==>", statement)

  #tutur
  match = re.search(r"tutur (.*)", sentence)
  if match:
    words = match.group(1).strip()
    if len(words.split()) == 1:
      people = words.split()[0]
    else:
      people = people

    statement = sentence.split(match.group(0))[0]
    print(people, "==>", statement)

  #menurutnya
  match = re.search(r"menurutnya,", sentence)
  if match:
    start_index = match.end()
    end_index = len(sentence)
    statement = sentence[start_index:end_index].strip()
    print(people, "==>", statement)

  #ungkap
  word = "ungkap"
  match = re.search(r"\b" + word + r"\b", sentence)
  if match:
    start = match.start()
    statement= sentence[:start]
    people = re.split(r"\W+", sentence[start+len(word):])[1]
    print(people, "==>", statement)

  #imbuh
  match = re.search(r"imbuh (.*)", sentence)
  if match:
    people = match.group(1).split(" ")[0]
    if people.endswith(","):
      people = people[:-1]
    statement = sentence.split(match.group(0))[0]
    print(people, "==>", statement)

  #ujar
  match = re.search(r"ujar (.*)", sentence)
  if match:
    if '.' in match.group(1).split(" ")[0]:
      if match.group(1).split(" ")[0].split(".")[0] == 'dia':
        people = people
      else:
        people = match.group(1).split(" ")[0].split(".")[0]
    else:
      people = match.group(1).split(" ")[0]

    if 'akun' in people:
      people = " ".join(match.group(1).split(" ")[:2])

    if people.endswith("."):
      people = people[:-1]

    statement = sentence.split(match.group(0))[0]
    print(people, "==>", statement)

  # #mengungkapkan
  # match = re.search(r"(?<=mengungkapkan ).+?(?=\.)", sentence)
  # if match:
  #   people = sentence[:match.start()].replace("mengungkapkan", "").strip()
  #   statement = match.group().strip()
  #   print(people, "==>", statement)

  #mengungkapkan
  match = re.search(r"(?<=mengungkapkan ).+?(?=\.)", sentence)
  if match:
    people = ' '.join(sentence[:match.start()].replace("mengungkapkan", "").strip().split()[:2])
    statement = match.group().strip()
    print(people, "==>", statement)

  #tutup
  match = re.search(r"\btutup\b (.*)", sentence)
  if match:
    words = match.group(1).strip()
    if len(words.split()) == 1:
      people = words.split()[0]
    else:
      people = people

    if '.' in people:
      split_titik = people.split(".")
      people = split_titik[0].strip()

    statement = sentence.split(match.group(0))[0]
    print(people, "==>", statement)

  #sambungnya
  match = re.search(r"sambungnya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

  #tambahnya
  match = re.search(r"tambahnya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

  #pungkas
  match = re.search(r"\bpungkas\b", sentence)
  if match:
    people = sentence.split(match.group(0))[1].strip()
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

  #katanya
  match = re.search(r"katanya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()
    print(people, "==>", statement)


  #pungkasnya
  match = re.search(r"pungkasnya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

  #menuturkan
  match = re.search(r'([a-zA-Z]+(?:\s+[a-zA-Z]+)?)\s+menuturkan,\s+(.*)', sentence)
  if match:
    if 'pun' in match.group(1):
      people = people

    else:

      if match.group(1) == 'ia':
        people = peopleW
      else:
        people = match.group(1)
        if len(people.split()) == 1:
          people = people.split()[0]

    statement = match.group(2)
    print(people, "==>", statement)

  #beber
  match = re.search(r'(.*)\bbeber\b\s(\w+)\s(.*)', sentence)
  if match:
      statement = match.group(1).strip()
      people = match.group(2)
      print(people, "==>", statement)
  else:
      pass

  #berharap
  match = re.search(r'(\w+)\sberharap,\s(.*)', sentence)
  if match:
    people = match.group(1)
    statement = match.group(2).strip()
    print(people, "==>", statement)
  else:
    pass

  #tandas
  match = re.search(r"\btandas\b", sentence)
  if match:
    index = match.start()
    statement = sentence[:index].strip()
    people = sentence[index+len("tandas"):].strip()
    print(people, "==>", statement)
  else:
    pass

  #sebut
  match = re.search(r"\bsebut\b", sentence)
  if match:
    index = match.start()
    statement = sentence[:index].strip()
    people = sentence[index+len("sebut"):].strip()
    print(people, "==>", statement)
  else:
    pass

  #menyebutkan
  match = re.search(r"\bmenyebutkan\b", sentence)
  if match:
    if sentence[:match.start()].strip() == 'ia':
      people = people
    else:
      people = sentence[:match.start()].strip()

    statement = sentence[match.end()+2:].strip()

    if len(people)>20:
      people = ''
      statement = ''

    print(people, "==>", statement)

  #imbuhnya
  match = re.search(r"imbuhnya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

  #tambah
  match = re.search(r"\btambah\b", sentence)
  if match:
    index = match.start()
    statement = sentence[:index].strip()

    if 'dia' in sentence[index+len("tambah"):].strip():
      people = people
    else:
      people = sentence[index+len("tambah"):].strip()
    print(people, "==>", statement)
  else:
    pass

  #ucapnya
  match = re.search(r"ucapnya", sentence)
  if match:
    people = people
    statement = sentence.split(match.group(0))[0].strip()

    print(people, "==>", statement)

satria budi ==> , gempa bumi berkekuatan 4,3 magnitudo terletak pada koordinat 7.27 ls dan 107.73 bt, atau tepatnya berlokasi di darat pada jarak 19 kilometer barat daya kabupaten garut pada kedalaman 3 kilometer.
satria budi ==> "berdasarkan rilis bmkg, dampak gempa bumi yang digambarkan peta tingkat guncangan (shakemap) bmkg dan berdasarkan laporan dari masyarakat dirasakan di wilayah garutâ pasirwangi, cisurupan, bayongbong, samarang dengan skala intensitas iv mmi. dengan memperhatikan lokasi episenter dan kedalaman hiposenternya, gempabumi yang terjadi merupakan jenis gempabumi dangkal akibat aktivitas sesar garsela,"
," ujarnya. ==> "secara umum untuk di kecamatan samarang lebih sedikit tapi kondisinya berat dan sedang, tapi di pasirwangi jumlahnya memang banyak 370 tapi ringan dan sedang. jadi, sekarang ini sedang melakukan verifikasi dulu dan personil yang melakukan assessment di lapangan, kita panggil di bpbd dan nanti akan melakukan pembahasan sehingga nanti tindaklanjut akan 

### kesini untuk hasil

In [ ]:
# coba stopwords di people agar string selain nama di remove

people_statement = {'people' : 'statement nya apa',
                    'luhut juga' : 'bahwa indonesia sedang berupaya untuk mengembangkan dan memperluas industri hilir, agar bisa memenuhi target menjadi produsen baterai lithium terbesar di dunia.'}

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
new_people_statement = {}
# Menghapus stopwords dari kunci dictionary
for key, value in people_statement.items():
    words = key.split()
    filtered_words = [word for word in words if word not in stop_words]
    new_key = ' '.join(filtered_words)
    new_people_statement[new_key] = value

print(new_people_statement)

{'people': 'statement nya apa', 'luhut': 'bahwa indonesia sedang berupaya untuk mengembangkan dan memperluas industri hilir, agar bisa memenuhi target menjadi produsen baterai lithium terbesar di dunia.'}


# ** gak dipakai... 2. Ambil kalimat yang di apit oleh tanda kutip tanda koma  dan titik**

In [ ]:
quotes = []

for x,y in enumerate(sentences):

  # print('Untuk kumpulan kalimat pada paragraf', x)

  quoted_strings = re.findall(r'“([^"]*)”', y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        # print('-', y)
      else:
        quotes.append(string)
        # print('-', string)

  quoted_strings = re.findall(r'"([^"]*)"', y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        # print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r"""'([^"]*)'""", y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r"""'([^']*)'""", y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r""",([^']*).""", y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r',([^.]+)[.]', y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r'\.([^.]+)[.]', y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)

  quoted_strings = re.findall(r'\-([^.]+)[.]', y)
  for string in quoted_strings:
      if len(string) < 30:
        quotes.append(y)
        #print('-', y)
      else:
        quotes.append(string)
        #print('-', string)



  # print('------------------------------'*100)

quotes = list(set(quotes))

for i, q in enumerate(quotes):
  print(i, '-', q)

0 - PKB menggelar forum Ijtima Ulama Nusantara pada 13-14 Januari 2023. Forum tersebut menghasilkan sejumlah keputusan penting bagi partai tersebut. Diantaranya adalah menegaskan pencalonan Muhaimin Iskandar sebagai Capres atau Cawapres dan penetapan pasangan calon maksimal pada Maret mendatang.
1 - Saat ditanya lebih lanjut soal hasil forum Ijtima Ulama Nusantara yang menyebutkan PKB harus menetapkan pasangan Capres-Cawapres maksimal Maret mendatang, dia pun hanya bungkam. Prabowo hanya memberikan kode jempol sambil pergi memasuki kendaraannya.
2 -  “Prinsipnya, Prabowo menghargai dan mempertimbangkan serius hasil Ijtima Ulama Nusantara
3 -  elektabilitasnya menurut sejumlah lembaga survei masih sangat rendah
4 - ” ujar Muzani. ”Tapi intinya, di bulan puasa pada pertengahan Maret sudah terkonsolidasikan semua.
5 -  Mereka yang akan membicarakan hal tersebut,” ujar Muzani
6 - Harapannya kan seperti itu, Prabowo yang jadi capres dan Cak Imin (Muhaimin) yang menjadi wapres,” kata Muzani 

In [ ]:
len(quotes)

31

Split menggunakan kalimat yang didapat diatas kemudian search tipe statement yang ada, misalnya 'kata' split dan cari apakah ada string 'kata' pada kalimat setelah di split, jika ada maka jadikan setelah kata itu people dan kalimat yg di ekstrak diatas menjadi statement nya. begitu juga yang lain.

**dibawah ini untuk generate statement & people**

In [ ]:
quotes = sentences

In [ ]:
import re

for x,y in enumerate(quotes):
  match = re.search(r'kata', y.lower())
  if match:
      print("ditemukan dalam text_mentah")
      print(x)
      print(y)
  else:
      print("tidak ditemukan dalam text_mentah")

tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah
ditemukan dalam text_mentah
2
"Ya kita Ikuti semua perkembangan," kata Prabowo saat ditemui di Sekretariat Bersama Gerindra-PKB di kawasan Menteng, Jakarta Pusat, Senin, 23 Januari 2023.
tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah
ditemukan dalam text_mentah
7
“Harapannya kan seperti itu, Prabowo yang jadi capres dan Cak Imin (Muhaimin) yang menjadi wapres,” kata Muzani saat ditemui secara terpisah di acara peresmian Sekber Gerindra-PKB. “Prinsipnya, Prabowo menghargai dan mempertimbangkan serius hasil Ijtima Ulama Nusantara.”
tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah
tidak ditemukan dalam text_mentah


In [ ]:
len(quotes)

31

In [ ]:
quotes[2]

'"Ya kita Ikuti semua perkembangan," kata Prabowo saat ditemui di Sekretariat Bersama Gerindra-PKB di kawasan Menteng, Jakarta Pusat, Senin, 23 Januari 2023.'

In [ ]:
re.search(r'kata (\w+)', text_mentah.split(quotes[6])[1][0:20])

In [ ]:
for i in range(len(quotes)):

  match = re.search(r'kata (\w+)', text_mentah.split(quotes[i])[1][0:20])
  if match:
      people = match.group(1)
      print(people, '==>', quotes[i])

Prabowo ==> Ya kita Ikuti semua perkembangan,


In [ ]:
q = quotes

In [ ]:
quotes = []

for i in q:
  # print(i)
  quotes.append(i.lower())

In [ ]:
quotes

['pkb menggelar forum ijtima ulama nusantara pada 13-14 januari 2023. forum tersebut menghasilkan sejumlah keputusan penting bagi partai tersebut. diantaranya adalah menegaskan pencalonan muhaimin iskandar sebagai capres atau cawapres dan penetapan pasangan calon maksimal pada maret mendatang.',
 'saat ditanya lebih lanjut soal hasil forum ijtima ulama nusantara yang menyebutkan pkb harus menetapkan pasangan capres-cawapres maksimal maret mendatang, dia pun hanya bungkam. prabowo hanya memberikan kode jempol sambil pergi memasuki kendaraannya.',
 ' “prinsipnya, prabowo menghargai dan mempertimbangkan serius hasil ijtima ulama nusantara',
 ' elektabilitasnya menurut sejumlah lembaga survei masih sangat rendah',
 '” ujar muzani. ”tapi intinya, di bulan puasa pada pertengahan maret sudah terkonsolidasikan semua.',
 ' mereka yang akan membicarakan hal tersebut,” ujar muzani',
 'harapannya kan seperti itu, prabowo yang jadi capres dan cak imin (muhaimin) yang menjadi wapres,” kata muzani sa

In [ ]:
text_mentah

'TEMPO.CO, Jakarta - Ketua Umum Partai Gerindra, Prabowo Subianto, belum dapat memastikan dirinya bersama Ketua Umum Partai Kebangkitan Bangsa (PKB), Muhaimin Iskandar, akan maju pada Pemilihan Presiden (Pilpres) 2024. Meskipun telah menerima hasil forum Ijtima Ulama Nusantara yang digelar PKB pertengahan bulan lalu, Prabowo masih belum mau memastikan Muhaimin sebagai pendampingnya.\nPrabowo menyatakan masih akan mempertimbangkan hasil forum tersebut. Dia pun irit bicara soal peluang Muhaimin menjadi pendampingnya pada Pilpres 2024.\n"Ya kita Ikuti semua perkembangan," kata Prabowo saat ditemui di Sekretariat Bersama Gerindra-PKB di kawasan Menteng, Jakarta Pusat, Senin, 23 Januari 2023.\nSaat ditanya lebih lanjut soal hasil forum Ijtima Ulama Nusantara yang menyebutkan PKB harus menetapkan pasangan Capres-Cawapres maksimal Maret mendatang, dia pun hanya bungkam. Prabowo hanya memberikan kode jempol sambil pergi memasuki kendaraannya.\nPKB menggelar forum Ijtima Ulama Nusantara pada 13

In [ ]:
output_people = []

for i in range(len(quotes)):

  match = re.search(r'ucap (\w+)', text_mentah.split(quotes[i])[1][0:20])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'kata (\w+)', text_mentah.split(quotes[i])[1][0:20])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'ujarnya', text_mentah.split(quotes[i])[1][0:30])
  if match:
      # people = match.group(1)
      print(people, '==>', quotes[i])

  match = re.search(r'tutur (\w+ \w+ \w+)', text_mentah.split(quotes[i])[1][0:30])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'ujar (\w+)', text_mentah.split(quotes[i])[1][0:20])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'(\w+) mengatakan', text_mentah.split(quotes[i])[0][-20:-1])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'(\w+) menyebut', text_mentah.split(quotes[i])[0][-20:-1])
  if match:
      people = match.group(1)
      output_people.append(people)
      print(people, '==>', quotes[i])

  match = re.search(r'tegasnya', text_mentah.split(quotes[i])[1][0:30])
  if match:
      # people = match.group(1)
      print(people, '==>', quotes[i])

  match = re.search(r'sambungnya', text_mentah.split(quotes[i])[1][0:30])
  if match:
      # people = match.group(1)
      print(people, '==>', quotes[i])

  match = re.search(r'sambungnya', text_mentah.split(quotes[i])[1][0:30])
  if match:
      # people = match.group(1)
      print(people, '==>', quotes[i])

IndexError: ignored

In [ ]:
people = 'dia'
if people == 'dia':
  print('bisa')

In [ ]:
count = Counter(output_people)
mode = count.most_common(1)[0][0]
print(mode)

In [ ]:
output_people

In [ ]:
print(re.search(r'mengakatan', text_mentah.split(quotes[-1])[0][-20:-1]))

In [ ]:
text_mentah.split(quotes[-1])[0][-20:-1]

In [ ]:
match = re.search(r'kata (\w+)', text_mentah.split(quotes[0])[1][0:12])
if match:
    people = match.group(1)
    print(people, '==>', quotes[0])

match = re.search(r'ujarnya', text_mentah.split(quotes[1])[1][0:30])
if match:
    # people = match.group(1)
    print(people, '==>', quotes[1])

match = re.search(r'tutur (\w+ \w+ \w+)', text_mentah.split(quotes[2])[1][0:30])
if match:
    people = match.group(1)
    print(people, '==>', quotes[2])

# match = re.search(r'ujar (\w+)', text_mentah.split(quotes[2])[1][0:20])
# if match:
#     people = match.group(1)
#     print(people, '==>', quotes[3])

In [ ]:
import pandas as pd

# Menentukan titik pembagian
split_index = df[df['text'] == 'kata'].index[0]

# Membagi DataFrame menjadi dua bagian
df_part1 = df.iloc[:split_index].reset_index(drop=True)
df_part2 = df.iloc[split_index:].reset_index(drop=True)

print(df_part1)
print(df_part2)

In [ ]:
df_part1

In [ ]:
text_mentah

In [ ]:
df_part1[df_part1['text'] == '“']

# print(df_part2['text'][1], '=>', df_part1)

In [ ]:
' '.join(df_part1['text'])

In [ ]:
dt = []
dl = []
statement = {}

for t,l in zip(df['text'].tolist(), df['label'].tolist()):
  # print(t, '-->', l)
  dt.append(t)
  dl.append(l)

  if l == 'PERSON':
    print(type(t))
    print(t, 'mengatakan : ', ' '.join(str(txt) for txt in text_mentah.lower().split(t)[0].split(',')[:-1]))
    # statement = {t : ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]))}

    # if bool(re.search(r"kata", text_mentah.lower())) == True:
    #   # print(t)
    #   statement[t] = ' '.join(str(txt) for txt in text_mentah.lower().split("kata")[0].split(',')[0:]).strip()
    #   # split = ' '.join(str(txt) for txt in text_mentah.lower().split(t)[1].split(',')[:-1]).strip()
    #   split_index = df[df['text'] == 'kata'].index[0]
    #   statement = df.iloc[:split_index].reset_index(drop=True)
    #   people = df.iloc[split_index:].reset_index(drop=True)



    # # elif bool(re.search(r"memaparkan", text_mentah.lower())) == True:
    # #   statement[t] = text_mentah.lower().split('memaparkan,')[1].strip()

    # # break

output = {dt[i]: dl[i] for i in range(len(dt))}
# print(text_mentah.lower())
print(statement)

In [ ]:
print(text_mentah.lower())

In [ ]:
dt = []
dl = []
statement = {}

for t,l in zip(df['text'].tolist(), df['label'].tolist()):
  # print(t, '-->', l)
  dt.append(t)
  dl.append(l)

  if l == 'PERSON':
    # print(type(t))
    # print(t, 'mengatakan : ', ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]))
    # statement = {t : ' '.join(str(txt) for txt in text_new.lower().split(t)[0].split(',')[:-1]))}

    if bool(re.search(r"kata", text_mentah.lower())) == True:
      # print(t)
      statement[t] = ' '.join(str(txt) for txt in text_mentah.lower().split("kata")[0].split(',')[0:]).strip()
      # split = ' '.join(str(txt) for txt in text_mentah.lower().split(t)[1].split(',')[:-1]).strip()


    # elif bool(re.search(r"memaparkan", text_mentah.lower())) == True:
    #   statement[t] = text_mentah.lower().split('memaparkan,')[1].strip()

    # break

output = {dt[i]: dl[i] for i in range(len(dt))}
# print(text_mentah.lower())
print(statement)

In [ ]:
print(dt)

In [ ]:
print(dh)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df[df.label == 'PERSON']

In [ ]:
" ".join(df['text'])

In [ ]:
import pandas as pd

# Buat DataFrame sederhana
df = pd.DataFrame({'kalimat': ['ini adalah contoh kalimat sederhana','ini adalah contoh kalimat sederhana yang tidak ada kata contoh']})

# Mengecek apakah kata 'contoh' ada dalam data dari kolom 'kalimat'
df['has_contoh'] = df['kalimat'].str.contains("contoh")

# Membagi data dalam kolom 'kalimat' berdasarkan kata 'contoh' jika kolom 'has_contoh' True
df.loc[df['has_contoh'] == True, 'kalimat_split'] = df.loc[df['has_contoh'] == True]['kalimat'].str.split('contoh')

# Tampilkan DataFrame
print(df)


In [ ]:
df['text'][0]

In [ ]:
import pandas as pd

# Buat DataFrame sederhana
df = pd.DataFrame({'value': ['part1-part2','part3-part4','part5-part6']})

# Membagi data dalam kolom 'value' berdasarkan karakter '-'
df['value'].str.split('-',expand=True)

In [ ]:
' '.join(df['text'].tolist()).split('louis van gaal')

In [ ]:
text_mentah.lower().split('louis van gaal')

In [ ]:
# coba

for i in range(len(df)):
  if df['label'][i] == 'PERSON':
    print(text_mentah.lower().split(df['text'][i]))



    for x in text_mentah.lower().split(df['text'][i]):
      print(x)

    break

In [ ]:
print(kata)

In [ ]:
text_mentah.lower()

In [ ]:
l_text = text_mentah.lower().split()

In [ ]:
print(l_text)

In [ ]:
' '.join(l_text)

In [ ]:
for i, v in enumerate(l_text):

  if v == 'kata':
    print(' '.join(l_text).split(v)[1])

    print(len(' '.join(l_text).split(v)))

In [ ]:
for y in range(len(hasil)):

  if hasil['words'][y] == 'kata':
    # print(hasil['words'].tolist())
    # print(y)
    hasil_kata = hasil.iloc[y:]
    # print(hasil_kata)

# print(hasil_kata['words'].tolist())
for i in range(hasil_kata.index[0], hasil_kata.index[-1]+1):
  # print(i)

  try:
    if hasil_kata['label'][i]=='B-PERSON' and hasil_kata['label'][i+1]!='I-PERSON':
      print(hasil_kata['words'][i], '=> PERSON')
    else:
      pass

  except:
    if hasil_kata['label'][i]=='B-PERSON' and ((i+1)==len(hasil_kata)):
      print(hasil_kata['words'][i], '=> PERSON')
    else:
      pass

  try:
    if hasil_kata['label'][i]=='B-PERSON' and hasil_kata['label'][i+1]=='I-PERSON' and hasil_kata['label'][i+2]=='I-PERSON':
      print(hasil_kata['words'][i],hasil_kata['words'][i+1],hasil_kata['words'][i+2], '=> PERSON')
    elif hasil_kata['label'][i]=='B-PERSON' and hasil_kata['label'][i+1]=='I-PERSON':
      print(hasil_kata['words'][i],hasil_kata['words'][i+1], '=> PERSON')
    else:
      pass

  except:
    pass

# print(' '.join([str(elem) for elem in text]))
# print('-'*100)

for x,y in enumerate(text):
  if bool(re.findall(r"(?=("+'|'.join(kata)+r"))", y)) == True:
    #print(y)

    if y == 'kata':
      print(' '.join([str(elem) for elem in text]).split('kata')[0].strip())

In [ ]:
print(' '.join([str(elem) for elem in text]))
print('-'*100)

for x,y in enumerate(text):
  if bool(re.findall(r"(?=("+'|'.join(kata)+r"))", y)) == True:
    #print(y)

    if y == 'kata':
      print(' '.join([str(elem) for elem in text]).split('kata')[0].strip())